### STML: Step-by-Step Code

#### Step 1: Connect to Nanonis

In [1]:
### Load modules ### & ### Establish Connection to Nanonis ###
import ipywidgets as ipy
import importlib
import os
import time
import sys
from IPython.display import display
import warnings
import pprint
sys.path.append("..")

# To install the light field python environment on a new machine:
try:
    import clr
except ImportError:
    !py -m pip install pythonnet

# Bypass normal STML Class and load testing environment
stml_testing_enviroment_active = False


# Import custom modules
import nanonis_connect_base
importlib.reload(nanonis_connect_base)
if stml_testing_enviroment_active:
    import stml_testing_enviroment
    importlib.reload(stml_testing_enviroment)
    print("STML Testing Environment Active")
else:
    import lightfield_base
    importlib.reload(lightfield_base)
    import datalogger_base
    importlib.reload(datalogger_base)
    import spe_read
    importlib.reload(spe_read)

### Establish Connection to Nanonis ###
nbase = nanonis_connect_base.nanonis_connect_base()
nbase_widgets = nbase.nanonis_conntect_widgets

# In case there is already a connection to Nanonis, use different ports
nbase.port_text.value = str(6503)
nbase.port2_text.value = str(6504)
nbase.port2_active.value = True
display(nbase_widgets)

# Establish connection
nbase.activate_connection_to_nanonis(None)

# Get current_session_path
current_session_path = nbase.utilities.SessionPathGet()
print(current_session_path)

TCP/IP connection with Nanonis established
Test: Get Bias from Nanonis:  -2.299999952316284 V
K:\Labs205\Labs\THz-STM\raw data\amsp\Porphyrin\725a_DiPhenaZnPor_H3_01\01_725a_NaCl_Ag111VT


#### Step 2: Connect to Lightfield

In [2]:
### Start Lightfield ###
if stml_testing_enviroment_active:
    raise Exception('Lightfield is not available in the STML testing environment.')

# Define lightfield experiment to load and data directory
experimentDirectory = r'C:\Users\Lab205.EMPA\Documents\LightField\Experiments'
experimentName = 'Ag111_VT.lfe'
dataDirectory = current_session_path

# Start Lightfield and connect to the selected experiment
lf = lightfield_base.lightField_connect(experimentDirectory,experimentName,dataDirectory=dataDirectory)
lf.open_lightField()

# Set filename to a default value with automatic numbering
lf.set_file_save_name('spec')
lf.change_file_save_add_time_bool(True)
lf.set_data_directory_external(dataDirectory)

# Set the lightfield data directory to the current Nanonis session path
def update_light_field_data_directory(change):
    current_nanonis_session_path = nbase.utilities.SessionPathGet()
    current_light_field_data_directory = lf.dataDirectory
    if current_nanonis_session_path != current_light_field_data_directory:
        print('Updating LightField data directory to: ', current_nanonis_session_path)
        lf.set_data_directory_external(current_nanonis_session_path)
    else:
        print('LightField data directory is already up to date.')

button_update_lightfield_datadirectory = ipy.Button(description='Update LightField Data Directory', layout=ipy.Layout(width='300px', height='30px'))
button_update_lightfield_datadirectory.on_click(update_light_field_data_directory)
display(button_update_lightfield_datadirectory)

# Available functions for Lightfield:
# lf.set_grating(grating_index) # 0: 150 g/mm, 800 nm blaze; 1: 600 g/mm, 750 nm blaze; 2: 1200 g/mm, 750 nm blaze
# lf.set_exposure_time(exposure_time) # in ms
# lf.set_grating_center_wavelength(center_wavelength) # in nm


LightField App Opened


Button(description='Update LightField Data Directory', layout=Layout(height='30px', width='300px'), style=Butt…

#### Step 3: Load basic interaction functions (Nanonis & Lightfield)

In [20]:
### STML Acquisition Class and Multi-Tool Acquire Button ###

# <----------------- Change Parameters Here ------------------------>
FILENAME_INCREMENT = True
ADDITIONAL_METADATA_DICT = None # {"Sample": "MoS2"} # This will be added to the metadata for all measurements
ONE_TIME_ADDITIONAL_METADATA_DICT = None # {"Filter":"LP575, SP1030", "PL>Laser": "SHG 515nm, 40MHz, 0.8mW"} # {"Sample": "MoS2"} # This will be added to the metadata only for the measurement executed within the current cell
# <----------------------------------------------------------------->

if stml_testing_enviroment_active:
    import stml_testing_enviroment
    importlib.reload(stml_testing_enviroment)
else:
    import stml_base
    importlib.reload(stml_base)
from IPython.display import clear_output, display
import threading
import ipywidgets as ipy

### Check that all necessary variables are defined and create STML Base Class
if 'nbase' not in globals():
    raise ValueError("Nanonis Base Class not defined.")
if stml_testing_enviroment_active:
    lf = stml_testing_enviroment.lightfield()
    stmlbase = stml_testing_enviroment.STMLBase(nbase,lf,additional_metadata_dict=ADDITIONAL_METADATA_DICT)
    print("STML Testing Enviroment Active")
else:
    if 'lf' not in globals():
        raise ValueError("LightField Base Class not defined.")
    stmlbase = stml_base.STMLBase(nbase,lf,additional_metadata_dict=ADDITIONAL_METADATA_DICT)



class acquire_spectra():

    def __init__(self,nbase,stmlbase,lf,acquire_settings=None, ONE_TIME_ADDITIONAL_METADATA_DICT=None):

        self.nbase = nbase
        self.lf = lf
        self.stmlbase = stmlbase

        self.FILENAME_INCREMENT = FILENAME_INCREMENT
        self.ONE_TIME_ADDITIONAL_METADATA_DICT = ONE_TIME_ADDITIONAL_METADATA_DICT

        # Default acquire settings
        acquire_settings_default = {
                "Experiment Type": "STML", # "STML" or "PL" or "TipPrep"
                "Repetitions": 1,
                "Tip Preparation Voltage (V)": 2.5,
                "Tip Preparation Current (pA)": 100,
                "Tip Preparation Exposure Time (ms)": 10000,
                "Laser Wavelength": 'None', # 'None', '405nm', '515nm', '640nm'
                "Laser Power": 0, # 0-100%
                "Filter": ('None',), # 'None', 'LP575', 'LP650', 'SP1030'
                "Slit Width": 200,
                "Experiment Comment": None}
        
        if acquire_settings is None:
            self.acquire_settings = acquire_settings_default
        else:
            self.acquire_settings = acquire_settings
            for key in acquire_settings_default.keys():
                if key not in self.acquire_settings.keys():
                    self.acquire_settings[key] = acquire_settings_default[key]

        self._create_widgets()

    def _create_widgets(self):

        relevant_settings_per_experiment_type = {
            "TipPrep": ["Tip Preparation Voltage (V)", "Tip Preparation Current (pA)", "Tip Preparation Exposure Time (ms)", "Repetitions", "Slit Width", "Experiment Comment", "Acquire Button"],
            "STML": ["Repetitions", "Slit Width", "Experiment Comment", "Acquire Button"],
            "Simple STML": ["Acquire Button"],
            "PL": ["Laser Wavelength", "Laser Power", "Filter", "Repetitions", "Slit Width", "Experiment Comment", "Acquire Button"],
        }
        relevant_settings = relevant_settings_per_experiment_type.get(self.acquire_settings["Experiment Type"], [])

        self.widgets = []

        self.experiment_setting_dropdown = ipy.Dropdown(options=['STML','PL','TipPrep'],
                                                value=self.acquire_settings["Experiment Type"],
                                                description="Experiment Type:",
                                                layout=ipy.Layout(width='350px',height='30px'),
                                                style={'description_width': 'initial'})
        self.widgets.append(self.experiment_setting_dropdown)
        self.experiment_setting_dropdown.observe(self._rebuild_ui, names='value')

        for setting in relevant_settings:

            if setting == "Tip Preparation Voltage (V)":
                self.tip_prep_voltage_input = ipy.FloatText(value=self.acquire_settings["Tip Preparation Voltage (V)"], description='Tip Prep. Voltage (V):',
                                                layout=ipy.Layout(width='350px',height='30px'),
                                                style={'description_width': 'initial'})
                self.widgets.append(self.tip_prep_voltage_input)
                self.tip_prep_voltage_input.observe(self._update_settings_libary_and_acquire_button, names='value')

            elif setting == "Tip Preparation Current (pA)":
                self.tip_prep_current_input = ipy.FloatText(value=self.acquire_settings["Tip Preparation Current (pA)"], description='Tip Prep. Current (pA):',
                                            layout=ipy.Layout(width='350px',height='30px'),
                                            style={'description_width': 'initial'})
                self.widgets.append(self.tip_prep_current_input)
                self.tip_prep_current_input.observe(self._update_settings_libary_and_acquire_button, names='value')

            elif setting == "Tip Preparation Exposure Time (ms)":
                self.tip_prep_exposure_time_input = ipy.FloatText(value=self.acquire_settings["Tip Preparation Exposure Time (ms)"], description='Tip Prep. Exposure Time (ms):',
                                            layout=ipy.Layout(width='350px',height='30px'),
                                            style={'description_width': 'initial'})
                self.widgets.append(self.tip_prep_exposure_time_input)
                self.tip_prep_exposure_time_input.observe(self._update_settings_libary_and_acquire_button, names='value')

            elif setting == "Laser Wavelength":
                self.laser_wavelength_dropdown = ipy.Dropdown(options=['None','405nm','515nm','640nm'],
                                                        value=self.acquire_settings["Laser Wavelength"],
                                                        description="Laser Wavelength:",
                                                        layout=ipy.Layout(width='350px',height='30px'),
                                                        style={'description_width': 'initial'})
                self.widgets.append(self.laser_wavelength_dropdown)
                self.laser_wavelength_dropdown.observe(self._update_settings_libary_and_acquire_button, names='value')
            
            elif setting == "Laser Power":
                self.laser_intensity_percentage = ipy.IntSlider(value=self.acquire_settings["Laser Power"], min=0, max=100, step=1, description='Laser Intensity (%)',
                                                            layout=ipy.Layout(width='350px',height='30px'),
                                                            style={'description_width': 'initial'})
                self.widgets.append(self.laser_intensity_percentage)
                self.laser_intensity_percentage.observe(self._update_settings_libary_and_acquire_button, names='value')
            
            elif setting == "Filter":
                self.filter_multiselect = ipy.SelectMultiple(options=['None','LP575','LP650','SP1030'],
                                            value=self.acquire_settings["Filter"],
                                            description="Filter:",
                                            layout=ipy.Layout(width='350px',height='100px'),
                                            style={'description_width': 'initial'})
                self.widgets.append(self.filter_multiselect)
                self.filter_multiselect.observe(self._update_settings_libary_and_acquire_button, names='value')

            elif setting == "Repetitions":
                self.repetitions_IntText = ipy.IntText(value=self.acquire_settings["Repetitions"], description='Repetitions:',
                                                    layout=ipy.Layout(width='350px',height='30px'),
                                                    style={'description_width': 'initial'})
                self.widgets.append(self.repetitions_IntText)
                self.repetitions_IntText.observe(self._update_settings_libary_and_acquire_button, names='value')

            elif setting == "Slit Width":
                self.slight_width_slider = ipy.IntSlider(value=self.acquire_settings["Slit Width"], min=0, max=200, step=1, description='Slight Width (um)',
                                            layout=ipy.Layout(width='350px',height='30px'),
                                            style={'description_width': 'initial'})
                self.widgets.append(self.slight_width_slider)
                self.slight_width_slider.observe(self._update_settings_libary_and_acquire_button, names='value')

            elif setting == "Experiment Comment":
                experiment_comment = self.acquire_settings.get("Experiment Comment", "")
                self.experiment_comment_text = ipy.Text(value=experiment_comment, description='Exp. Comment:',
                                        layout=ipy.Layout(width='350px',height='30px'),
                                        style={'description_width': 'initial'})
                self.widgets.append(self.experiment_comment_text)
                self.experiment_comment_text.observe(self._update_settings_libary_and_acquire_button, names='value')
            
            elif setting == "Acquire Button":
                self.acquire_single_stml_button = ipy.Button(description='Acquire Single STML Spectrum',
                                                    layout=ipy.Layout(width='350px',height='30px'))
                self.widgets.append(self.acquire_single_stml_button)
                self.acquire_single_stml_button.on_click(self.acquire_single_thread_start)

        # Update the settings library and acquire button based on the current settings
        self._update_settings_libary_and_acquire_button(None)

        self.log_output = ipy.Textarea(
            value='',
            layout=ipy.Layout(width='350px', height='200px'),
            style={'description_width': 'initial'}
        )

        # Display the widgets
        display(ipy.HBox([ipy.VBox(self.widgets),self.log_output]))

    def _update_settings_libary_and_acquire_button(self,change):
        """ Update the settings library and acquire button based on the current values of the widgets. """
        
        # Update the self.acquire_settings dictionary with the current values from the widgets
        if hasattr(self, 'experiment_setting_dropdown'):
            self.acquire_settings["Experiment Type"] = self.experiment_setting_dropdown.value
        if hasattr(self, 'tip_prep_voltage_input'):
            self.acquire_settings["Tip Preparation Voltage (V)"] = self.tip_prep_voltage_input.value
        if hasattr(self, 'tip_prep_current_input'):
            self.acquire_settings["Tip Preparation Current (pA)"] = self.tip_prep_current_input.value
        if hasattr(self, 'tip_prep_exposure_time_input'):
            self.acquire_settings["Tip Preparation Exposure Time (ms)"] = self.tip_prep_exposure_time_input.value
        if hasattr(self, 'laser_wavelength_dropdown'):
            self.acquire_settings["Laser Wavelength"] = self.laser_wavelength_dropdown.value
        if hasattr(self, 'laser_intensity_percentage'):
            self.acquire_settings["Laser Power"] = self.laser_intensity_percentage.value
        if hasattr(self, 'filter_multiselect'):
            self.acquire_settings["Filter"] = self.filter_multiselect.value
        if hasattr(self, 'repetitions_IntText'):
            self.acquire_settings["Repetitions"] = self.repetitions_IntText.value
        if hasattr(self, 'slight_width_slider'):
            self.acquire_settings["Slit Width"] = self.slight_width_slider.value
        if hasattr(self, 'experiment_comment_text'):
            self.acquire_settings["Experiment Comment"] = self.experiment_comment_text.value

        # Update the acquire button description based on the experiment type and repetitions
        no_repeatitions = self.repetitions_IntText.value
        if self.acquire_settings["Experiment Type"] == "STML":
            if no_repeatitions > 1:
                self.acquire_single_stml_button.description = f'Acquire {no_repeatitions} STML Spectra'
            else:
                self.acquire_single_stml_button.description = 'Acquire STML Spectrum'

        if self.acquire_settings["Experiment Type"] == "PL":
            if no_repeatitions > 1:
                self.acquire_single_stml_button.description = f'Acquire {no_repeatitions} Spectra'
            else:
                self.acquire_single_stml_button.description = 'Acquire Spectrum'

        if self.acquire_settings["Experiment Type"] == "TipPrep":
            TARGET_BIAS = self.tip_prep_voltage_input.value
            TARGET_CURRENT = self.tip_prep_current_input.value * 1e-12
            TARGET_EXPOSURE_TIME = self.tip_prep_exposure_time_input.value  # in ms
            description = 'Record STML @ '+str(round(TARGET_BIAS,1))+'V & '+str(round(TARGET_CURRENT*1e12))+'pA & '+str(round(TARGET_EXPOSURE_TIME/1000))+'s. Then reset.'
            self.acquire_single_stml_button.description = description
            self.acquire_single_stml_button.icon = 'bolt'
            self.acquire_single_stml_button.button_style = 'danger'
        else:
            self.acquire_single_stml_button.icon = 'check'
            self.acquire_single_stml_button.button_style = ''

    def _rebuild_ui(self, change):
        """Update the widgets based on the selected experiment type."""

        self.acquire_settings["Experiment Type"] = self.experiment_setting_dropdown.value

        # Clear current widgets
        for widget in self.widgets:
            widget.close()
            self.log_output.close()

        self._create_widgets()

    def iprint(self,message):
        """Print a message to the log output."""
        self.log_output.value += message + '\n'

    def acquire_spectra_for_all_experiments(self):
        """Acquire spectra for all experiments based on the current settings."""
        # Clear the log output
        self.log_output.value = ''

        # Update the settings library for pulling settings during acquisition
        self._update_settings_libary_and_acquire_button(None)

        # Determine Filename
        if self.FILENAME_INCREMENT:
            filename = self.stmlbase.interative_filename_generator(filebase="stml_",fileending=".dat",other_expected_filebases_endings=None)
            if filename is None or os.path.exists(os.path.join(current_session_path, filename)):
                filename = time.strftime("%Y%m%d_%H%M%S")
        else:
            filename = time.strftime("%Y%m%d_%H%M%S")

        # Create a dictionary for one-time additional metadata
        if self.ONE_TIME_ADDITIONAL_METADATA_DICT is None:
            one_time_additional_metadata_dict = {}
        else:
            one_time_additional_metadata_dict = self.ONE_TIME_ADDITIONAL_METADATA_DICT.copy()
        if hasattr(self, 'experiment_setting_dropdown'):
            one_time_additional_metadata_dict["Experiment"] = self.acquire_settings["Experiment Type"]
        if hasattr(self, 'laser_wavelength_dropdown'):
            one_time_additional_metadata_dict["Prima>Laser Wavelength (nm)"] = self.acquire_settings["Laser Wavelength"]
        if hasattr(self, 'laser_intensity_percentage'):
            one_time_additional_metadata_dict["Prima>Intensity (%)"] = self.acquire_settings["Laser Power"]
        if hasattr(self, 'filter_multiselect'):
            one_time_additional_metadata_dict["Filter"] = str(self.acquire_settings["Filter"])
        if hasattr(self, 'slight_width_slider'):
            one_time_additional_metadata_dict["Slight Width (um)"] = self.acquire_settings["Slit Width"]
        if hasattr(self, 'experiment_comment_text'):
            if self.acquire_settings["Experiment Comment"] != '':
                one_time_additional_metadata_dict["Experiment Comment"] = self.acquire_settings["Experiment Comment"]

        ## Handle the different experiment types
        current_experiment_type = self.acquire_settings["Experiment Type"]

        ## STML Spectrum Acquisition
        if current_experiment_type == "STML":
            filename0 = filename
            if filename0.endswith('.dat'):
                filename0 = filename0[:-4]
            for i in range(self.acquire_settings["Repetitions"]):
                if self.acquire_settings["Repetitions"] > 1:
                    filename = filename0 + '_rep' + str(i)
                else:
                    filename = filename0 
                # Acquire STML Spectrum
                self.iprint(f"Acquiring STML Spectrum: {filename}.dat")
                self.stmlbase.aquire_stml(filename,spe_data_return=False,print_out=True,one_time_additional_metadata_dict=one_time_additional_metadata_dict,logger=self.iprint)
                self.iprint("Finished.")

        ## Tip Preparation
        if current_experiment_type == "TipPrep":

            self.iprint("Starting Tip Preparation...")

            # Get original bias, current setpoint and exposure time
            original_bias = self.nbase.biasmodule.Get()
            original_setpoint_current = self.nbase.zcontroller.SetpntGet()
            original_exposure_time = lf.get_exposure_time()

            self.iprint(f"Changing Bias to {round(self.acquire_settings['Tip Preparation Voltage (V)'],2)} V, Current Setpoint to {round(self.acquire_settings['Tip Preparation Current (pA)'])} pA, and Exposure Time to {round(self.acquire_settings['Tip Preparation Exposure Time (ms)']/1000)} s.")

            # Set new bias, current setpoint and exposure tim
            nbase.biasmodule.Set(self.acquire_settings["Tip Preparation Voltage (V)"])
            nbase.zcontroller.SetpntSet(self.acquire_settings["Tip Preparation Current (pA)"] * 1e-12)
            lf.set_exposure_time(self.acquire_settings["Tip Preparation Exposure Time (ms)"])
            time.sleep(0.5)

            filename0 = filename
            if filename0.endswith('.dat'):
                filename0 = filename0[:-4]
            for i in range(self.acquire_settings["Repetitions"]):
                if self.acquire_settings["Repetitions"] > 1:
                    filename = filename0 + '_rep' + str(i)
                else:
                    filename = filename0
                # Acquire STML Spectrum
                self.iprint(f"Acquiring STML Spectrum: {filename}.dat")
                spe_data = self.stmlbase.aquire_stml(filename,spe_data_return=True,print_out=False,one_time_additional_metadata_dict=one_time_additional_metadata_dict)
                self.iprint("Finished.")

            # Wait for the exposure time + 1 second
            # wait_time = lf.get_exposure_time() / 1000 + 1
            # time.sleep(wait_time)
            time.sleep(0.5)

            # Reset the bias, current setpoint and exposure time
            self.iprint("Resetting Bias, Current Setpoint and Exposure Time to original values.")
            self.nbase.biasmodule.Set(original_bias)
            self.nbase.zcontroller.SetpntSet(original_setpoint_current)
            lf.set_exposure_time(original_exposure_time)

        ## PL Spectrum Acquisition
        if current_experiment_type == "PL":
            filename0 = filename
            if filename0.endswith('.dat'):
                filename0 = filename0[:-4]
            for i in range(self.acquire_settings["Repetitions"]):
                if self.acquire_settings["Repetitions"] > 1:
                    filename = filename0 + '_rep' + str(i)
                else:
                    filename = filename0
                self.iprint(f"Acquiring Spectrum: {filename}.dat")
                # Acquire Spectrum
                spe_data = self.stmlbase.aquire_stml(filename,spe_data_return=True,print_out=False,one_time_additional_metadata_dict=one_time_additional_metadata_dict,logger=self.iprint)
                self.iprint("Finished.")

    def acquire_single_thread_start(self,change):
        """
        Threaded function to acquire single STML spectrum.
        """
        # Start the thread
        acquire_single_thread = threading.Thread(target=self.acquire_spectra_for_all_experiments)
        acquire_single_thread.start()
        acquire_single_thread.join()

if "acquire_spectra_initalized" in globals():
    acquire_settings = acquire_spectra_initalized.acquire_settings
else:
    acquire_settings = None
# Initialize the acquire_spectra class
acquire_spectra_initalized = acquire_spectra(nbase,stmlbase,lf,acquire_settings=acquire_settings, ONE_TIME_ADDITIONAL_METADATA_DICT=ONE_TIME_ADDITIONAL_METADATA_DICT)


Running DataLogger Channel Test...
All channels found in the datalogger file.
Datalogger channel test successfully performed.


#### Step 4: Experimental Sequencer: New Design

In [19]:
### Experimental Sequencer: New Design ###

import sequencer_nd
importlib.reload(sequencer_nd)
ExperimentalSequencer = sequencer_nd.ExperimentalSequencer

### Check that all necessary variables are defined
if 'nbase' not in globals(): warnings.warn("Nanonis Base Class not defined.")
if stml_testing_enviroment_active:
    lf = stml_testing_enviroment.lightfield()
else:
    if 'lf' not in globals(): warnings.warn("LightField Base Class not defined.")


### Start STML Base Class
if stml_testing_enviroment_active:
    lf = stml_testing_enviroment.lightfield()
    stmlbase = stml_testing_enviroment.STMLBase(nbase,lf,additional_metadata_dict=None)
else:
    if 'stmlbase' not in locals():
        stmlbase = stml_base.STMLBase(nbase,lf,additional_metadata_dict=None)

sequencer = ExperimentalSequencer(nbase,lf,stmlbase,INITIALIZE_NANONIS_MODULES_ONLY=False)
sequencer.EXPERIMENT_SLEEP_INTERVAL = 1  # time to wait after each experiment step (in seconds)


IndexError: list index out of range

In [ ]:
# Get current Experimental Sequence
pprint.pprint(sequencer.experiment_list)
grid_points = sequencer.experiment_list


In [ ]:
# Write new experimental sequence (Example)
sequencer.experiment_list = [('set_stml_center_wavelength', 600.0),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('set_stml_center_wavelength', 650.0),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('set_stml_center_wavelength', 695.0),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('set_stml_center_wavelength', 740.0),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('set_stml_center_wavelength', 790.0),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('set_stml_center_wavelength', 825.0),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('set_stml_center_wavelength', 875.0),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('set_stml_center_wavelength', 920.0),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('set_stml_center_wavelength', 965.0),
 ('single_point_stml', {}),
 ('single_point_stml', {}),
 ('single_point_stml', {}),]
sequencer.update_sequence_display()


##### Setting Sequences

#### Step 5: Grid STML Tool

In [ ]:
! py -m pip install opencv-python

In [ ]:
### STML Experiment Sequence: Scripted & Threaded: STML on Grid Points with XY SHIFT VECTOR CORRECTION ###
import ipywidgets as ipy
import time
import threading
import math
import numpy as np
import json
from scipy.ndimage import gaussian_filter
from skimage.registration import phase_cross_correlation
sys.path.append(r'./spmpy')
from spmpy import Spm
import cv2
from PIL import Image
import matplotlib.pyplot as plt

import atomtracker_base
importlib.reload(atomtracker_base)

class sequencer_stml_xyz_correction():

    """STML Grid Experiment Sequencer with Z Drift Stabilization and (X,Y) Shift Vector Correction by Image Correlation"""

    def __init__(self,nbase,lf,stmlbase,atbase,xy_correction_after_no=10,z_correction_after_no=2,point_list=None):
        """
        Input:
        -----
        - nbase: Nanonis Base Module
        - lf: Lightfield Module
        - stmlbase: STML Base Module
        - xy_correction_after_no: Number of STML Spectra Acquisitions after which the (X,Y) Shift Vector Correction is performed
        - z_correction_after_no: Number of STML Spectra Acquisitions after which the Z Drift Correction is performed
        - point_list: List of dictionaries containing the grid points in the following format (if None, the grid points are aquired from Nanonis):
            [{'grid_point_number':0,'x':0.0,'y':0.0,'filebase':'20210901_123456_0'},...]
        """
        ###########################################################################################################################################################################
        ########################################## ADD ANGLE CORRECTION FOR MASKED GRID SELECTION --> UNTIL THEN ONLY USE ANGLE == 0 DEG ########################################## 
        ###########################################################################################################################################################################
        # Necessary Interaction Classes
        self.nbase = nbase
        self.lf = lf
        self.stmlbase = stmlbase
        self.atbase = atbase

        # testing variables
        self._loop_test_disable_stm = False # use to disable stm actions 
        self._loop_test_disable_detector = False # use to disable lightfield

        # Internal Variables
        self.experimentType = 'stml'
        self.mask_fig,self.mask_fig_ax = plt.subplots()
        self.mask_fig.set_size_inches(3,3)
        self.mask_fig_ax.axis('off')
        self.xy_correction_after_no = xy_correction_after_no
        self.z_correction_after_no = z_correction_after_no
        self.filename_list = []
        self.grid_params = None
        self.grid_points_params = []
        self.stm_images = []
        self.ref_image = None
        self.ref_spm_image = None
        self.ref_image_dimensions = [None,None]
        self.ref_image_data = (None,None)
        self.ref_location = None
        self.scan_frame_info = self.nbase.scan.FrameGet() #[x,y,w,h,angle]
        self.xy_correction_array = []
        self.z_correction_array = []
        self.currentGridIndex = 0
        self.log_str = ''
        self.stop_event = threading.Event()
        self.stop_event.clear()
        self.pause_event = threading.Event()
        self.pause_event.clear()

        # Create Widgets
        self.create_widgets()
        with self.figure_display:
            plt.show(self.mask_fig)
        # Overwrite Grid Points if point_list is not None and has the correct format
        if point_list is not None and set(point_list[0].keys()) == {'grid_point_number','x','y','filebase'}:
            self.grid_points_params = point_list
            # Update Button Style
            self.grab_grid_points_button.style.button_color = 'lightgreen'
            self.grab_grid_points_button.description = 'Grid Points Aquired!'
            self.grab_grid_points_button.tooltip = 'Click to aquire a new grid points.'


    def create_widgets(self):
        #### experiment parameters
        self.measurementBias = ipy.FloatText(value=-2.8,
                                             description='Exp. Bias [V]:',
                                             tiptool='Bias voltage to use during measurements',
                                             disabled=False,
                                             layout=ipy.Layout(width='300px',height='30px'))
    
        self.zControlToggle = ipy.Checkbox(value=False,
                                            description='Z-Ctrl:',
                                            tiptool='Toggle Feedback for measurement',
                                            disabled=False,
                                            layout=ipy.Layout(width='148px',height='30px'))
        
        self.tipHeightOffset = ipy.FloatText(value=-240,
                                             description='Offset [pm]:',
                                             tiptool='tip lift offset to use during measurements',
                                             disabled=False,
                                             layout=ipy.Layout(width='148px',height='30px'))
        
        self.referenceBias = ipy.FloatText(value=1.0,
                                             description='Ref. Bias [V]:',
                                             tiptool='Bias voltage to use during drift correction',
                                             disabled=False,
                                             layout=ipy.Layout(width='300px',height='30px'))
        
        self.referenceCurrent = ipy.FloatText(value=1e-11,
                                             description='Ref. I [A]:',
                                             tiptool='Setpoint current to use during drift correction',
                                             disabled=False,
                                             layout=ipy.Layout(width='300px',height='30px'))
        
        self.thresholdToggle = ipy.Checkbox(value=False,
                                            description='I-Ctrl:',
                                            tiptool='Toggle Current Threshold for measurement',
                                            disabled=False,
                                            layout=ipy.Layout(width='148px',height='30px'))

        self.thresholdCurrent = ipy.FloatText(value=5e-12,
                                               description = 'Current [A]:',
                                                tiptool='Current threshold for STML measurement',
                                                disabled=False,
                                                layout=ipy.Layout(width='148px',height='30px'))
        
        self.xyDriftInterval = ipy.IntText(value=10,
                                           description='XY Count [N]:',
                                           tiptool='Interval between XY Drift Corrections',
                                           disabled=False,
                                           layout=ipy.Layout(width='300px',height='30px'))
        
        self.zDriftInterval = ipy.IntText(value=4,
                                          description='Z Count [N]:',
                                          tiptool='Interval between Z Drift Corrections',
                                          disabled=False,
                                          layout=ipy.Layout(width='300px',height='30px'))
        
        self.maskThreshold = ipy.BoundedFloatText(value=.5,
                                                  min = 0.001,
                                                  max = 1.0,
                                                  step = 0.05,
                                            description='Level [%]:',
                                            tiptool='Threshold for the mask creation',
                                            disabled=False,
                                            layout=ipy.Layout(width='148px',height='30px'))
        
        self.maskType = ipy.Dropdown(options=['I','z'],
                                     value = 'I',
                                        description='Type:',
                                        tiptool='Select which channel to use for mask creation',
                                        disabled=False,
                                        layout=ipy.Layout(width='148px',height='30px'))
        
        self.maskSelection = ipy.Dropdown(options=['None'],
                                            value='None',
                                            description='Mask Image:',
                                            tiptool='Select the mask to use for grid point selection',
                                            disabled=False,
                                            layout=ipy.Layout(width='248px',height='30px'))
        
        self.gridStartIndex = ipy.IntText(value=0,
                                          description='Start Index:',
                                          tiptool='Start position within the grid',
                                          disabled=False,
                                          layout=ipy.Layout(width='300px',height='30px'))
        self.experimentSelection = ipy.Dropdown(options=['None'],
                                                value='None',
                                                description='File:',
                                                tiptool='Select the experiment file to load',
                                                disabled=False,
                                                layout=ipy.Layout(width='148px',height='30px'))
        self.resume_experiment_toggle = ipy.Checkbox(value=False,
                                                    description='Resume:',
                                                    tiptool='Toggle to resume the experiment from the last grid point',
                                                    disabled=False,
                                                    layout=ipy.Layout(width='100px',height='30px'))
        
        self.resume_experiment_toggle.observe(self.resume_experiment_toggled,names='value')

        self.experimentSelection_button = ipy.Button(description='',
                                                     icon = 'refresh',
                                                    tiptool='Refresh the experiment list',
                                                    layout=ipy.Layout(width='48px',height='30px'))
        self.experimentSelection_button.on_click(self.find_experiments)

        #### gui buttons and display widgets
        self.run_masked_grid_button = ipy.Button(description='',
                                                icon='spinner',
                                                tooltip='Generated a cloud of grid points based on the image and threshold',
                                                layout=ipy.Layout(width='300px',height='30px'))
        self.run_masked_grid_button.on_click(self.run_masked_grid)

        self.maskListRefresh = ipy.Button(description='',
                                          icon='refresh',
                                          tiptool='Refresh the mask list',
                                          layout=ipy.Layout(width='48px',height='30px'))
        self.maskListRefresh.on_click(self.refresh_mask_list)

        self.figure_display = ipy.Output(layout=ipy.Layout(width='300px',height='300px'))

        self.grab_ref_location_button = ipy.Button(description='Acquire Reference Location',
                                            tiptool='Aquire current follow me location (x,y,V,I) to be used as ref location',
                                            layout=ipy.Layout(width='300px',height='30px'))
        self.grab_ref_location_button.on_click(self.grab_ref_location)

        self.grab_grid_points_button = ipy.Button(description='Acquire Grid Points',
                                            tiptool='Aquire (& Calculate) Grid Points Parameters from Nanonis',
                                            layout=ipy.Layout(width='300px',height='30px'))
        self.grab_grid_points_button.on_click(self.grab_grid_points)

        self.grab_ref_image_button = ipy.Button(description='Acquire Reference Image',
                                            tiptool='Acquire reference image',
                                            layout=ipy.Layout(width='300px',height='30px'))
        self.grab_ref_image_button.on_click(self.grab_ref_image)

        self.pause_button = ipy.ToggleButton(description='',
                                       icon='pause',
                                       tooltip='Pause / resume Experiment',
                                       layout=ipy.Layout(width='47px',height='30px'),
                                       disabled=False)

        self.pause_button.observe(self.pause_experiment, names='value')

        self.start_button = ipy.Button(description='',
                                       icon='play',
                                       tooltip='Start Experiment',
                                       layout=ipy.Layout(width='47px',height='30px'),
                                       disabled=False)
        self.start_button.on_click(self.start_experiment)

        self.stop_button = ipy.Button(description='',
                                      icon='stop',
                                      tooltip='Stop Experiment',
                                      layout=ipy.Layout(width='47px',height='30px'),
                                      disabled=True)
        self.stop_button.on_click(self.stop_experiment)

        
        self.log_textarea = ipy.Textarea(value='',layout=ipy.Layout(width='300px',height='300px'))

        self.progressBar = ipy.IntProgress(value=0,
                                           min = 0,
                                           max = 100,
                                           description='Progress:',
                                           bar_style='info',
                                           orientation='horizontal',
                                           layout=ipy.Layout(width='300px',height='30px'))
        #### layouts
        self.maskImageHbox = ipy.HBox([self.maskSelection,self.maskListRefresh])
        self.maskThresholdHbox = ipy.HBox([self.maskThreshold,self.maskType])
        self.zHbox = ipy.HBox([self.zControlToggle,self.tipHeightOffset])
        self.thresholdHbox = ipy.HBox([self.thresholdToggle,self.thresholdCurrent])
        self.experimentHbox = ipy.HBox([self.resume_experiment_toggle,self.experimentSelection,self.experimentSelection_button])
        self.maskVbox = ipy.VBox([self.maskImageHbox,
                                  self.maskThresholdHbox,
                                  self.run_masked_grid_button,
                                  self.figure_display],
                                  layout=ipy.Layout(width='320px'))
        
        self.parametersVbox = ipy.VBox([self.referenceBias,
                                        self.referenceCurrent,
                                        self.measurementBias,
                                        self.zHbox,
                                        self.thresholdHbox,
                                        self.xyDriftInterval,
                                        self.zDriftInterval,
                                        self.gridStartIndex,
                                        self.experimentHbox],layout=ipy.Layout(width='320px'))
        
        self.guiVbox = ipy.VBox([self.grab_ref_location_button,
                                self.grab_ref_image_button,
                                self.grab_grid_points_button,
                                ipy.HBox([self.pause_button,self.start_button,self.stop_button]),
                                self.log_textarea,
                                self.progressBar],layout=ipy.Layout(width='320px'))
        
        self.widgets = ipy.HBox([self.guiVbox,self.parametersVbox,self.maskVbox])
        display(self.widgets)
    
    ### WIDGETS CALLS ###
    def resume_experiment_toggled(self,change):
        if self.resume_experiment_toggle.value:
            if self.experimentSelection.value == 'None':
                self.resume_experiment_toggle.value = False
                self.log_print('Please select an experiment file to resume the experiment.')
            else:
                self.refresh_mask_list('a')
                self.read_json(os.path.join(nbase.utilities.SessionPathGet(),self.experimentSelection.value))
    
    def find_experiments(self,change):
        files = os.listdir(nbase.utilities.SessionPathGet())
        experiment_files = [file for file in files if '.json' in file]
        self.experimentSelection.options = experiment_files
        self.experimentSelection.value = self.experimentSelection.options[0]
    
    def run_masked_grid(self,change):
        """
        Runs the masked grid point generation based on the mask image and threshold
        Requirements:
        - maskSelection: image should acquired using the same scan frame as the reference image used for xy drift
        """
        current_session_path = nbase.utilities.SessionPathGet()
        image_filename = self.maskSelection.value
        mask_image = Spm(os.path.join(current_session_path,image_filename))
        ImgOrigin = mask_image.get_param("scan_dir")
        (mask_image_data,_) = mask_image.get_channel(channel=self.maskType.value,direction='forward',flatten=False,offset=False,zero=False)
        if ImgOrigin == 'down':
            mask_image_data = np.flip(mask_image_data,0)
        x,y,w,h,angle = nbase.scan.FrameGet()
        self.mask_image_info = {'spm':mask_image,'data':mask_image_data,'x':x,'y':y,'w':w,'h':h,'angle':angle}
        grid_points = self.create_masked_grid_points(mask_image_data,self.maskThreshold.value,x,y,w,h)
        print('Number of grid points selected:', len(grid_points))
        filebase = '20250521_174901'#time.strftime("%Y%m%d_%H%M%S")
        self.grid_points_params = []
        for i in range(len(grid_points)):
            self.grid_points_params.append({'grid_point_number':i,
                                            'x': grid_points[i][0],
                                            'y': grid_points[i][1],
                                            'ix':grid_points[i][2],
                                            'iy':grid_points[i][3],
                                            'filebase': filebase+"_"+str(i)})
        self.initial_grid_points_params = self.grid_points_params.copy()
        # Update Button Style
        self.grab_grid_points_button.style.button_color = 'lightgreen'
        self.grab_grid_points_button.description = 'Grid Points Aquired!'
        self.grab_grid_points_button.tooltip = 'Click to aquire a new grid points.'
        self.show_mask_selection()
        self.progressBar.max = len(self.grid_points_params)

    def refresh_mask_list(self,change):
        files = os.listdir(nbase.utilities.SessionPathGet())
        sxm_files = [file for file in files if '.sxm' in file]
        self.maskSelection.options = sxm_files
        self.maskSelection.value = self.maskSelection.options[-1]

    def grab_ref_image(self,change):
        current_session_path = nbase.utilities.SessionPathGet()
        # acquire image
        if change != 'a':
            self.log_print('Acquiring reference image...')
            self.nbase.scan.Action("start","up")
            self.nbase.scan.WaitEndOfScan()
            time.sleep(2)
            last_image_filename = self.last_acquired_sxm(current_session_path)
        else:
            last_image_filename = self.maskSelection.value
        #self.last_acquired_sxm(current_session_path)
        if last_image_filename is None:
            self.log_print('No recent image found in the current session folder.')
            raise ValueError('No recent image found in the current session folder.')
        else:
            self.stm_images.append(last_image_filename)
            self.ref_image = last_image_filename
        self.ref_spm_image = Spm(os.path.join(current_session_path,self.ref_image))
        self.ref_image_data = self.ref_spm_image.get_channel(channel='z',direction='forward',flatten=False,offset=False,zero=False); # data, unit
        ImgOrigin = self.ref_spm_image.get_param("scan_dir")
        if ImgOrigin == 'down':
            self.ref_image_data = (np.flip(self.ref_image_data[0],0),self.ref_image_data[1])
        py,px = self.ref_image_data[0].shape
        self.ref_image_dimensions = [py,px]
        print('Image dimensions (py,px):',py,px)
        self.scan_frame_info = self.nbase.scan.FrameGet() # returns [x,y,w,h,angle]
        print('Scan Frame Info [x,y,w,h,angle]:', self.scan_frame_info)
        self.grab_ref_image_button.style.button_color = 'lightgreen'
        self.log_print(f'Reference Image acquired: {self.ref_image}')

    def grab_ref_location(self,change):
        """Reads (x,y) from follow me and I Setpoint from zcontroller, V from Bias Module and saves them as reference location"""
        x_position, y_position, z_position = self.atbase.run_positioning(wait_time=30,duration=10,interval=0.1)
        #x_position, y_position = nbase.followme.XYPosGet(Wait_for_newest_data=True)
        #z_position = nbase.zcontroller.ZPosGet()
        i_setpoint = nbase.zcontroller.SetpntGet()
        bias = nbase.biasmodule.Get()

        self.ref_location = {'x': x_position, 'y': y_position, 'z': z_position, 'i': i_setpoint, 'v': bias}
        self.log_print(f'Reference Location grabbed: {self.ref_location}')
        # Update Button Style
        self.grab_ref_location_button.style.button_color = 'lightgreen'
        self.grab_ref_location_button.description = 'Reference Location Aquired!'
        self.grab_ref_location_button.tooltip = 'Click to aquire a new reference location.'

    def grab_grid_points(self,change):
        """Reads Grid Parameters from Nanonis and calculates the grid points based on the grid parameters"""
        number_x_points, number_y_points, center_x, center_y, width, height, angle = self.nbase.pattern.GridGet()
        print('Grid Parameters aquired: ', number_x_points, number_y_points, center_x, center_y, width, height, angle)
        # calculates a regular grid based on nanonis grid parameters
        grid_points = self.calculate_grid_points(number_x_points, number_y_points, center_x, center_y, width, height, angle)
        filebase = time.strftime("%Y%m%d_%H%M%S")
        for i in range(len(grid_points)):
            self.grid_points_params.append({'grid_point_number':i,'x': grid_points[i][0], 'y': grid_points[i][1], 'filebase': filebase+"_"+str(i)})

        # Update Button Style
        self.grab_grid_points_button.style.button_color = 'lightgreen'
        self.grab_grid_points_button.description = 'Grid Points Aquired!'
        self.grab_grid_points_button.tooltip = 'Click to aquire a new grid points.'

        self.progressBar.max = len(self.grid_points_params)

    def start_experiment(self,change):
        self.log_print('Starting Experiment Sequence...')
        self.start_button.disabled = True
        self.stop_button.disabled = False
        self.run_masked_grid_button.disabled = True
        self.progressBar.value = 0
        self.referenceBias.value = self.nbase.biasmodule.Get()
        self.referenceCurrent.value = self.nbase.zcontroller.SetpntGet()
        self.stop_event.clear()
        self.stmlbase.AQUIRE_LOCK = False
        self.experiment_thread = threading.Thread(target=self.experiment_loop)
        self.experiment_thread.start()
        #self.experiment_thread.join()

    def experiment_finished(self):
        self.stop_button.disabled = True
        if not self._loop_test_disable_stm:
            self.move_tip_for_measurement('drift')
        self.write_json()
        self.start_button.disabled = False
        self.run_masked_grid_button.disabled = False


    def stop_experiment(self,change):
        self.log_print('Stopping Experiment Sequence...')

        # Check if Scan is running via second port
        if self.nbase.scan2.StatusGet() == 1:
            self.nbase.scan2.Action("stop")

        # Update Button Style
        self.start_button.disabled = False
        self.stop_button.disabled = True
        self.stop_event.set()

        # Print Completed File List
        #print(f"{self.filename_list=}")

        # <------------------------------------------------------------------ UPDATE STOP BEHAVIOUR HERE ------------------------------------------------------------------>
        self.experiment_finished()
        # <---------------------------------------------------------------------------------------------------------------------------------------------------------------->
    def pause_experiment(self,change):
        if change['new'] == True:
            # Pause the experiment
            self.pause_event.set()
            self.pause_button.style.icon = 'spinner'
            self.pause_button.tooltip = 'Click to continue experiment'
            self.log_print(f'pause experiment: {self.getTimeStamp()}')
        if change['new'] == False:
            # Resume the experiment
            self.pause_event.clear()
            self.pause_button.style.icon = 'pause'
            self.pause_button.tooltip = 'Click to pause experiment'
            self.log_print(f'continue experiment: {self.getTimeStamp()}')

    ### HELPER FUNCTIONS ###
    def getTimeStamp(self):
        year,month,day,hour,minute,second,*_ = time.localtime()
        return f'{hour}:{minute}:{second}'
    
    def display_grid_in_nanonis(self):
        nPoints = len(self.grid_points_params)
        Xarray = [self.grid_points_params[i]['x'] for i in range(nPoints)]
        Yarray = [self.grid_points_params[i]['y'] for i in range(nPoints)]
        self.nbase.pattern.CloudSet(1,nPoints,Xarray,Yarray)
    
    def show_mask_selection(self):
        """ Display selected mask image and the point cloud generated from run_masked_grid"""
        ax = self.mask_fig_ax
        ax.clear()
        ax.axis('off')
        ImgOrigin = 'lower'
        if self.mask_image_info['spm'].get_param("scan_dir") == "down":
            ImgOrigin = "upper"
        ax.imshow(self.mask_image_info['data'],origin=ImgOrigin,cmap='viridis')
        marker_x = [grid_point['ix'] for grid_point in self.grid_points_params]
        marker_y = [grid_point['iy'] for grid_point in self.grid_points_params]
        colorArray = []
        for m in range(len(marker_x)):
            if m >= self.currentGridIndex:
                color = 'r'
            else:
                color = 'b'
            colorArray.append(color)
        #print(marker_y)
        #print(marker_x)

        ax.scatter(marker_x,marker_y,
                s=8,marker='+',c=colorArray)
        self.mask_fig.tight_layout()
        self.mask_fig.canvas.draw()
        self.progressBar.description = f'0/{len(self.grid_points_params)}'
        if not self.start_button.disabled:
            self.display_grid_in_nanonis()

    def create_masked_grid_points(self,image_data,mask_threshold,center_x, center_y, width, height):
        """
        Converts pixel data into array of grid points for pixel data above a certain threshold
        
        Input:
        -----
        - image_data: 2D array of either current or height data
        - mask_threshold: selection criteria for grid point (e.g., minimum current)
        - center_x: center x position of the image in m
        - center_y: center y position of the image in m
        - width: width of the image in m
        - height: height of the image in m

        Output:
        ------
        - grid_points: list of grid points (x[m],y[m],x_index,y_index)
        """
        image_data = gaussian_filter(image_data,2)
        if self.maskType.value == 'I':
            image_data = abs(image_data)
        if self.maskType.value == 'z':
            image_data -= np.min(image_data)
        image_data /= np.max(image_data)
        py,px = image_data.shape
        dx = width / px # pixel width
        dy = height / py # pixel height
        # origin position (bottom left corner)
        X = center_x - width/2
        Y = center_y - height/2
        # 0,0 pixel coordinates
        x0 = X + dx/2
        y0 = Y + dy/2
        coordinates = lambda index: [x0 + index[1]*dx, y0 + index[0]*dy]
        index_list = np.ndindex(image_data.shape)
        masked_grid_points = []
        for i,index in enumerate(index_list):
            if abs(image_data[index]) > abs(mask_threshold):
                masked_grid_points.append([*coordinates(index),index[1],index[0]])
        self.grab_ref_image('a')
        return masked_grid_points

    def calculate_grid_points(self,number_x_points, number_y_points, center_x, center_y, width, height, angle):
        """
        Calculates the grid points based on the grid parameters.

        Input:
        -----
        - number_x_points: number of points in x direction
        - number_y_points: number of points in y direction
        - center_x: center x position of the grid in m
        - center_y: center y position of the grid in m
        - width: width of the grid in m
        - height: height of the grid in m
        - angle: angle of the grid in degrees

        Output:
        ------
        - grid_points: list of grid points (x[m],y[m],x_index,y_index)
        """

        grid_points = []
        
        # Calculate the step size for x and y directions
        step_x = width / (number_x_points)
        step_y = height / (number_y_points)
        
        # Convert the angle to radians
        angle_rad = math.radians(angle)
        
        # Calculate the starting position of the grid
        start_x = center_x - (width / 2)
        start_y = center_y - (height / 2)
        
        for i in range(number_y_points+1):
            for j in range(number_x_points+1):
                # Calculate the current position based on the grid index
                current_x = start_x + j * step_x
                current_y = start_y + i * step_y
                
                # Rotate the current position around the center point
                rotated_x = center_x + (current_x - center_x) * math.cos(angle_rad) - (current_y - center_y) * math.sin(angle_rad)
                rotated_y = center_y + (current_x - center_x) * math.sin(angle_rad) + (current_y - center_y) * math.cos(angle_rad)
                
                # Add the rotated position to the grid points
                grid_points.append((rotated_x, rotated_y,j,i))
        
        return grid_points
    
    def last_acquired_sxm(self,filepath):
        """Returns the filename of the last sxm file in the session folder."""
        files = os.listdir(filepath)
        files = [file for file in files if file.endswith('.sxm')]
        files_sorted = sorted(files, key=lambda x: os.path.getctime(os.path.join(filepath, x)))
        if len(files_sorted) > 0:
            return files_sorted[-1]
        else:
            return None

    def log_print(self,text):
        if self.log_str == '':
            self.log_str = text
        else:
            self.log_str = self.log_str + '\n' + text
        self.log_textarea.value = self.log_str
    
    def write_json(self):
        # get directory name as file label
        # experiment parameters
        experimentType = 'stml'
        sessionPath = self.sessionPath
        ### reference info
        reference_image = self.ref_image
        reference_location = self.ref_location # includes position and setpoint data
        referenceBias = self.referenceBias.value
        referenceCurrent = self.referenceCurrent.value
        ### measurement info
        measurementBias = self.measurementBias.value
        zControlToggle = self.zControlToggle.value
        tipHeightOffset = self.tipHeightOffset.value
        thresholdToggle = self.thresholdToggle.value
        thresholdCurrent = self.thresholdCurrent.value
        # drift correction
        xyDriftInterval = self.xyDriftInterval.value
        zDriftInterval = self.zDriftInterval.value
        xyDrift = self.xy_correction_array
        zDrift = self.z_correction_array
        # grid mask parameters
        maskThreshold = self.maskThreshold.value
        maskType = self.maskType.value
        maskSelection = self.maskSelection.value
        mask_image_info = self.mask_image_info.copy()
        mask_image_info.pop('spm')
        mask_image_info.pop('data')
        # grid point parameters
        final_grid_points = self.grid_points_params
        #self.run_masked_grid(None)
        initial_grid_points = self.initial_grid_points_params
        last_grid_point = self.currentGridIndex
        output_items = locals().copy()
        output_items.pop('self')
        with open(os.path.join(sessionPath,'experimentDetails.json'),'w') as f:
            json.dump(output_items,f,indent=1)

    def read_json(self,path):
        with open(path,'r') as f:
            loaded = json.load(f)
        self.sessionPath = loaded['sessionPath']
        self.experimentType = loaded['experimentType']
        self.ref_image = loaded['reference_image']
        self.ref_spm_image = Spm(os.path.join(self.sessionPath,self.ref_image))
        self.ref_image_data = self.ref_spm_image.get_channel(channel='z',direction='forward',flatten=False,offset=False,zero=False)
        self.ref_image_data = self.ref_spm_image.get_channel(channel='z',direction='forward',flatten=False,offset=False,zero=False)
        self.ref_image_dimensions = [self.ref_image_data[0].shape[1],self.ref_image_data[0].shape[0]]
        self.ref_location = loaded['reference_location']
        self.referenceBias.value = loaded['referenceBias']
        self.referenceCurrent.value = loaded['referenceCurrent']
        self.measurementBias.value = loaded['measurementBias']
        self.zControlToggle.value = loaded['zControlToggle']   
        self.tipHeightOffset.value = loaded['tipHeightOffset']
        self.thresholdToggle.value = loaded['thresholdToggle']
        self.thresholdCurrent.value = loaded['thresholdCurrent']
        self.xyDriftInterval.value = loaded['xyDriftInterval']
        self.zDriftInterval.value = loaded['zDriftInterval']
        self.xy_correction_array = loaded['xyDrift']
        self.z_correction_array = loaded['zDrift']
        self.maskThreshold.value = loaded['maskThreshold']
        self.maskType.value = loaded['maskType']
        self.maskSelection.value = loaded['maskSelection']
        self.mask_image_info = loaded['mask_image_info']
        maskSPM = Spm(os.path.join(self.sessionPath,self.maskSelection.value))
        (mask_image_data,_) = maskSPM.get_channel(channel=self.maskType.value,direction='forward',flatten=False,offset=False,zero=False)
        self.mask_image_info['spm'] = maskSPM
        self.mask_image_info['data'] = mask_image_data
        self.scan_frame_info = [self.mask_image_info['x'],self.mask_image_info['y'],self.mask_image_info['w'],self.mask_image_info['h'],self.mask_image_info['angle']]
        self.nbase.scan.FrameSet(*self.scan_frame_info[:-1],angle=self.scan_frame_info[-1])
        self.grid_points_params = loaded['final_grid_points']
        self.initial_grid_points_params = loaded['initial_grid_points']
        self.progressBar.max = len(self.grid_points_params)
        self.currentGridIndex = loaded['last_grid_point']
        self.gridStartIndex.value = self.currentGridIndex
        self.show_mask_selection()
        # update GUI elements
        self.grab_ref_image_button.style.button_color = 'lightgreen'
        self.grab_ref_location_button.style.button_color = 'lightgreen'
        self.grab_grid_points_button.style.button_color = 'lightgreen'
        self.log_print('Experiment Details Loaded from JSON File')

    def resume_experiment(self):
        #self.perform_z_drift_stabilization_at_reference_position()
        self.perform_xy_drift_correction_at_reference_position()
        self.perform_z_drift_stabilization_at_reference_position()

    def move_tip_for_measurement(self,measurement):
        if measurement not in ['drift','experiment']:
            raise ValueError('Invalid measurement type. Use "drift" or "experiment".')
        # esnure foodback is off
        self.nbase.zcontroller.OnOffSet(False)

        if measurement == 'drift':
            # check for z movement --> ensure tip is at highest point before moving / changing parameters
            if self.ref_location['z'] > self.nbase.zcontroller.ZPosGet():
                self.ramp_height_to_value(self.ref_location['z'])
            # move to reference xy position and set bias / feedback
            self.nbase.followme.XYPosSet(self.ref_location['x'],self.ref_location['y'],Wait_end_of_move=True)
            time.sleep(1)
            self.ramp_bias_to_value(self.ref_location['v'])
            self.ramp_current_to_value(self.ref_location['i'])
        elif measurement == 'experiment':
            experiment_location_z = self.z_correction_array[-1][1] + self.tipHeightOffset.value*1e-12
            heightChanged = False
            # check for z movement --> ensure tip is at highest point before moving / changing parameters
            if experiment_location_z > self.nbase.zcontroller.ZPosGet():
                self.ramp_height_to_value(experiment_location_z)
                heightChanged = True
            # set bias and let experiment loop move tip to next grid point
            self.ramp_bias_to_value(self.measurementBias.value)
            # move tip to current grid point
            i = self.currentGridIndex
            point_x = self.grid_points_params[i]['x']
            point_y = self.grid_points_params[i]['y']
            if not self._loop_test_disable_stm:
                self.nbase.followme.XYPosSet(point_x,point_y,Wait_end_of_move=True)
            if not heightChanged:
                self.ramp_height_to_value(experiment_location_z)

    def perform_z_drift_stabilization_at_reference_position(self):
        """Go to reference location, switch on Z-Controller, set current and bias, stablize Z-Drift for 10 sec and switch off Z-Controller"""
        self.log_print('Performing Z Drift Stabilization at Reference Position...')
        self.move_tip_for_measurement('drift')
        ### z stabilization by average height over 10 seconds
        tipHeights = []
        self.atbase.activate()
        time.sleep(30)
        for k in range(int(2//0.05)):
            tipHeights.append(self.nbase.zcontroller.ZPosGet())
            time.sleep(0.05)
        timestamp = self.getTimeStamp()
        self.z_correction_array.append([timestamp,np.average(tipHeights[len(tipHeights)//2:])]) # aveage the second half to cut out the initial drift
        self.atbase.deactivate()

    # Lateral Drift Compensation
    ### Phase cross Correlation --> good for non-rotation molecules
    def ppc_calculation(self,refData,newData,threshold=0.5):
        # image data must have origin in bottom left corner
        filtered_ref = gaussian_filter(refData,2)
        filtered_ref -= np.min(filtered_ref)
        filtered_ref = filtered_ref*(filtered_ref>threshold*abs(filtered_ref).max())
        filtered_data = gaussian_filter(newData,2)
        filtered_data -= np.min(filtered_data)
        filtered_data = filtered_data*(filtered_data>0.5*abs(filtered_data).max())
        shift_measured, error, diffphase = phase_cross_correlation(filtered_ref, filtered_data)
        dy,dx = shift_measured
        self.log_print(f'measured shift (py,px): {shift_measured}')
        return (-dy,-dx) # y,x :: dimensions match shape of input data
    ### Centroid estimation via cv2.moments
    def analyze_moment(self,data,threshold=0.8):
        ### set minimum value to zero
        data -= np.min(data)
        ### normalize data to 1
        data /= np.max(data)
        ### mask data according to threshold
        mask_data = np.uint8((data>threshold))
        ### calculate contours
        contours,hierarch = cv2.findContours(mask_data,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        moments = []
        centroids = []
        self.log_print(f"Number of contours: {len(contours)} :: Threshold should be set s.t. only one contour is found")
        for c in contours:
            M = cv2.moments(c)
            moments.append(M)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            centroids.append([cX,cY])
            self.log_print(f'Contour centroid: [{cX},{cY}]')
        return {'cX':centroids[0][0],'cY':centroids[0][1],'M':moments}
    def centroid_calculation(self,refData,newData,threshold=0.8):
        # image data must have origin in bottom left corner
        refMoment = self.analyze_moment(refData,threshold=threshold)
        newMoment = self.analyze_moment(newData,threshold=threshold)
        dx = newMoment['cX'] - refMoment['cX']
        dy = newMoment['cY'] - refMoment['cY']
        return (dy,dx)
    
    def perform_xy_drift_correction_at_reference_position(self):
        self.log_print('Performing XY Drift Correction at Reference Position...')
        self.move_tip_for_measurement('drift')
        
        # Check for stop criteria
        if self.stop_event.is_set():
            return 1
        # Perform XY Drift Correction
        x0,y0 = (self.ref_location['x'],self.ref_location['y'])
        x1,y1,z = self.atbase.run_positioning(wait_time=60,duration=5,interval=0.05)
        x_shift_nm = x1-x0
        y_shift_nm = y1-y0
        self.log_print(f'measured drift: {x_shift_nm}, {y_shift_nm}')
        
        if abs(x_shift_nm) > .1*self.scan_frame_info[2] or abs(y_shift_nm) > .1*self.scan_frame_info[3]:
            self.log_print(f'XY Drift Correction: Shifted by +{str(round(x_shift_nm*1e9,2))} nm,+{str(round(y_shift_nm*1e9,2))} nm')
            self.log_print('XY Drift Correction: Shift too large. XY Drift Compensation Failed')
            self.xy_drift_compensation_counter = 0
            self.z_drift_compensation_counter = 0
            raise ValueError('XY Drift Correction: Shift too large. XY Drift Compensation Failed')

        # Update Grid Point Parameters
        for j in range(self.currentGridIndex,len(self.grid_points_params)):
            self.grid_points_params[j]['x'] += x_shift_nm
            self.grid_points_params[j]['y'] += y_shift_nm
        
        # Update scan frame to compensate xy drift
        self.scan_frame_info = nbase.scan.FrameGet()
        print('Pre-corrected scan frame center (x,y):',self.scan_frame_info[:2])
        self.scan_frame_info[0] += x_shift_nm #self.scan_frame_info = nbase.Scan.FrameGet # returns [x,y,w,h,angle]
        self.scan_frame_info[1] += y_shift_nm
        self.nbase.scan.FrameSet(*self.scan_frame_info[:-1],angle=self.scan_frame_info[-1])
        print('Updated scan frame center (x,y):',self.nbase.scan.FrameGet()[:2])
        self.log_print('XY Drift Correction: Shifted by ('+str(round(x_shift_nm*1e12,3))+' pm,'+str(round(y_shift_nm*1e12,3))+' pm)')
        timestamp = self.getTimeStamp()
        self.xy_correction_array.append([x_shift_nm,y_shift_nm,timestamp])
        self.xy_drift_compensation_counter = 0

        # Update reference position X,Y
        self.ref_location['x'] = x1
        self.ref_location['y'] = y1

    def perform_xy_drift_correction(self):
        # Switch zcontroller only on at reference location
        self.nbase.followme.XYPosSet(self.ref_location['x'],self.ref_location['y'],Wait_end_of_move=True)
        time.sleep(1)
        self.ramp_bias_to_value(self.ref_location['v'])
        self.nbase.zcontroller.OnOffSet(True)
        self.nbase.zcontroller.SetpntSet(self.ref_location['i'])
        
        # Check for stop criteria
        if self.stop_event.is_set():
            return 1

        # Perform XY Drift Correction
        ## acquire drifted image
        self.nbase.zcontroller.OnOffSet(True)
        self.nbase.scan.Action("start","up")
        self.nbase.scan.WaitEndOfScan()
        self.nbase.zcontroller.OnOffSet(False)
        time.sleep(1)

        # grab drifted image filepath
        current_session_path = nbase.utilities.SessionPathGet()
        last_image_filename = self.last_acquired_sxm(current_session_path)
        self.stm_images.append(last_image_filename)
        
        # assign reference image values accordingly ####Load stm_image[-2] as image1
        (refData,chUnit) = self.ref_image_data

        # Load stm_image[-1] as image2
        image2 = Spm(os.path.join(current_session_path,self.stm_images[-1]))
        (chData2,chUnit) = image2.get_channel(channel='z',direction='forward',flatten=False,offset=False,zero=False);
        ImgOrigin = image2.get_param("scan_dir")
        if ImgOrigin == 'down':
            chData2 = np.flip(chData2,0)
        self.log_print('Image Correlation: '+str(self.ref_image)+' and '+str(self.stm_images[-1]))

        # Perform Image Correlation
        shift_measured = self.centroid_calculation(refData,chData2,threshold=float(self.maskThreshold.value))
        print('measured shift (py,px):',shift_measured)
        x_shift_nm = shift_measured[1]*self.scan_frame_info[2]/self.ref_image_dimensions[1] # scan frame width / image width in pixels
        y_shift_nm = shift_measured[0]*self.scan_frame_info[3]/self.ref_image_dimensions[0] # scan frame height / image height in pixels ,,,, self.scan_frame_info[1]/abs(self.scan_frame_info[1])*
        print('calculated drift:',x_shift_nm,y_shift_nm)
        
        if abs(x_shift_nm) > .2*self.scan_frame_info[2] or abs(y_shift_nm) > .2*self.scan_frame_info[3]:
            self.log_print('XY Drift Correction: Shifted by ('+str(round(x_shift_nm*1e9,2))+' nm,'+str(round(y_shift_nm*1e9,2))+' nm)')
            self.log_print('XY Drift Correction: Shift too large. XY Drift Compensation Failed')
            self.xy_drift_compensation_counter = 0
            self.z_drift_compensation_counter = 0
            raise ValueError('XY Drift Correction: Shift too large. XY Drift Compensation Failed')

        # Update Grid Point Parameters
        for j in range(self.currentGridIndex,len(self.grid_points_params)):
            self.grid_points_params[j]['x'] += x_shift_nm
            self.grid_points_params[j]['y'] += y_shift_nm
        
        # Update scan frame to compensate xy drift
        self.scan_frame_info = nbase.scan.FrameGet()
        print('Pre-corrected scan frame center (x,y):',self.scan_frame_info[:2])
        self.scan_frame_info[0] += x_shift_nm #self.scan_frame_info = nbase.Scan.FrameGet # returns [x,y,w,h,angle]
        self.scan_frame_info[1] += y_shift_nm
        self.nbase.scan.FrameSet(*self.scan_frame_info[:-1],angle=self.scan_frame_info[-1])
        print('Updated scan frame center (x,y):',self.nbase.scan.FrameGet()[:2])
        self.log_print('XY Drift Correction: Shifted by ('+str(round(x_shift_nm*1e12,3))+' pm,'+str(round(y_shift_nm*1e12,3))+' pm)')
        timestamp = self.getTimeStamp()
        self.xy_correction_array.append([x_shift_nm,y_shift_nm,timestamp])
        self.xy_drift_compensation_counter = 0

        # Update reference position X,Y
        self.ref_location['x'] += x_shift_nm
        self.ref_location['y'] += y_shift_nm

        # update grid points in nanonis
        self.display_grid_in_nanonis()

    ### ramp bias to between two values ####
    def ramp_bias_to_value(self,target_value):
        """Ramp bias between values at a rate of 2 V/s"""
        time.sleep(.1) # a slight pause before changing the bias
        # ramp bias
        currentBias = self.nbase.biasmodule.Get()
        targetBias = target_value
        # if change bias polarity, simply jump down to 0 first
        if np.sign(currentBias) != np.sign(targetBias):
            # ensure feedback is off
            if self.nbase.zcontroller.OnOffGet() == True:
                self.nbase.zcontroller.OnOffSet(False)
            else:
                pass
            self.nbase.biasmodule.Set(0)
            currentBias = 0
        if currentBias != targetBias:
            deltaV = targetBias - currentBias
            Nsteps = int(abs(deltaV)//0.1)
            biasSteps = np.linspace(currentBias,targetBias,np.max([Nsteps,2])) # ensures at least 2 steps are taken; needed to ensure linspace works
            for bias in biasSteps:
                self.nbase.biasmodule.Set(bias)
                time.sleep(0.05)
            self.nbase.biasmodule.Set(targetBias)
        else:
            pass
        self.log_print('Bias Set to Value: '+str(target_value)+' V')

    def ramp_height_to_value(self,target_value):
        """Ramp height between values at a rate of 100pm/s"""
        z0 = self.nbase.zcontroller.ZPosGet()
        dz = target_value - z0
        Nsteps = int(abs(dz)//10e-12)
        for i in range(Nsteps-1): # stop one step before final value to avoid overshoot
            z = z0 + (i+1)*dz/Nsteps
            self.nbase.zcontroller.ZPosSet(z)
            time.sleep(0.1)
        self.nbase.zcontroller.ZPosSet(target_value)
    
    def ramp_current_to_value(self,target_value,gain=5e-15):
        """ Engage feedback with reduced p-gain, slowing down controller response to reduce creep"""
        self.log_print('Ramping Current to Value: '+str(target_value)+' A')
        p0,t0,i0 = self.nbase.zcontroller.GainGet()
        self.nbase.zcontroller.GainSet(gain,t0)
        self.log_print('setting p-gain to: '+str(gain))
        self.nbase.zcontroller.SetpntSet(target_value)
        self.nbase.zcontroller.OnOffSet(True)
        for i in range(1000):
            time.sleep(0.1)
            current = self.nbase.currentmodule.Get()
            if abs(current - target_value) < abs(target_value*0.1):
                break
        self.log_print('reseting p-gain to: '+str(p0))
        self.nbase.zcontroller.GainSet(p0,t0)

    ### MAIN EXPERIMENT THREAD ###
    def experiment_loop(self):

        if self.ref_location is None:
            raise ValueError('Reference Location is not aquired. Please aquire the reference location first.')
        if self.grid_points_params == []:
            raise ValueError('Grid Points are not aquired. Please aquire the grid points first.')

        # Calculate the estimated experiment time
        exposure_time = self.lf.get_exposure_time()
        [x,y,w,h,angle] = self.nbase.scan.FrameGet()
        [fwd_speed,bwd_speed,fwd_line_time,bwd_line_time,const_param,speed_ratio] = self.nbase.scan.SpeedGet()
        [num_channels,channel_indexes,pixels,lines] = self.nbase.scan.BufferGet()
        time_estimate_scan = {'w':w,
                        'lines':lines,
                        'speed_ratio':speed_ratio,
                        'fwd_speed':fwd_speed,
                        'eta':w*lines*(1+speed_ratio)/fwd_speed}
        time_estimate_scan['eta'] = time_estimate_scan['w']*time_estimate_scan['lines']*(1+time_estimate_scan['speed_ratio'])/time_estimate_scan['fwd_speed']
        no_of_xy_drift_corrections = int(len(self.grid_points_params)/(self.xy_correction_after_no+1))
        total_eta_sec = time_estimate_scan['eta']*no_of_xy_drift_corrections+len(self.grid_points_params)*exposure_time/1000
        total_eta = time.gmtime(total_eta_sec)

        self.log_print('Estimated Experiment Time: '+str(total_eta.tm_hour)+'h '+str(total_eta.tm_min)+'m '+str(total_eta.tm_sec)+'s')
        self.log_print('Start Time: '+time.strftime("%Y-%m-%d %H:%M:%S"))
        self.log_print('End Time: '+time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time() + total_eta_sec)))

        # <----------------------------------------------------------------------- UPDATE EXPERIMENT HERE ----------------------------------------------------------------------->
    
        # Loop Description
        '''        
        --- pre-loop initialization ---
        1. Acquire check that reference image exists
        2. Go to reference location and stabilize Z-Drift with reference setpoint (I and V)
        3. Establish measurement height - with feedback off
        4. Ramp bias to measurement value
        5. Engage measurement loop
        --- measurement loop ---
        1. Go to grid position i
        2. Acquire STML Spectrum
        3. Check compensation counters
        3.a. if xy drift compensation counter is reached, acquire drifted image and perform image correlation
        3.b. Update Grid Point positions, scan image center position, reference position
        3.c. if z drift compensation counter is reached, stabilize z drift at reference position
        3.d. restablish measurement height and bias
        4. Repeat 1-3 for all grid points
        '''

        try:
            # Find last aquired image
            '''        
            if self.ref_image == None:
                self.log_print('No recent image found in the current session folder.')
                self.log_print('Please acquire a reference image')
                raise ValueError('No recent image found in the current session folder.')
 
            current_session_path = nbase.utilities.SessionPathGet()
            last_image_filename = self.last_acquired_sxm(current_session_path)
            if last_image_filename is None:
                self.log_print('No recent image found in the current session folder.')
                raise ValueError('No recent image found in the current session folder.')
            else:
                self.stm_images.append(last_image_filename)
            '''

            self.sessionPath = self.nbase.utilities.SessionPathGet()
            self.z_drift_compensation_counter = 0
            self.xy_drift_compensation_counter = 0
            doZDriftCorrection = False
            doXYDriftCorrection = False
            self.currentGridIndex = 0
            if self.resume_experiment_toggle.value == True:
                self.currentGridIndex = self.gridStartIndex.value
            # Perform Z Drift Stabilization at Reference Position
            if not self._loop_test_disable_stm:
                self.perform_z_drift_stabilization_at_reference_position() # goes to reference position, changes bias / setpoint, measures tip height sets measurement height w.r.t average recorded tip height
                self.log_print('Z Drift Stabilization Complete')
                self.perform_xy_drift_correction_at_reference_position() # goes to reference position, changes bias / setpoint, measures tip height sets measurement height w.r.t average recorded tip height
                self.log_print('XY Drift Stabilization Complete')
                self.log_print('Moving tip for experiment')
                self.move_tip_for_measurement('experiment')
                time.sleep(0.3) # slight pause before ramping the bias

            self.write_json() # write experiment details to json file
            # Start Experiment Loop
            for i in range(len(self.grid_points_params)):
                self.currentGridIndex = i
                # update progressbar
                self.progressBar.value = i+1
                self.progressBar.description = f'{i+1}/{len(self.grid_points_params)}'
                if i < self.gridStartIndex.value:
                    continue # skip to starting position
                else:
                    self.currentGridIndex = i
                # Check for stop criteria
                if self.stop_event.is_set():
                    print('stop event is_set')
                    break

                # Go to Location and Aquire STML Spectrum
                point_x = self.grid_points_params[i]['x']
                point_y = self.grid_points_params[i]['y']
                if not self._loop_test_disable_stm:
                    self.nbase.followme.XYPosSet(point_x,point_y,Wait_end_of_move=True)
                if doXYDriftCorrection == True or doZDriftCorrection == True or i == 0:
                    # under these conditions the tip is moving from farther away
                    time.sleep(1)
                else:
                    time.sleep(0.1)
                
                # check current 
                if self.thresholdToggle.value == True:
                    currents = []
                    for point in range(int(1.5//0.05)):
                        currents.append(self.nbase.currentmodule.Get())
                        time.sleep(0.05)
                    if abs(np.average(currents[len(currents)//2:]))<self.thresholdCurrent.value: # need gui element for current threshold
                        continue

                # Acquire STML Spectrum
                self.log_print('Aquire STML at Grid:'+str(i+1)+'/'+str(len(self.grid_points_params)))
                filename = self.grid_points_params[i]['filebase']
                if not self._loop_test_disable_detector:
                    try:
                        self.stmlbase.aquire_stml(filename,spe_data_return=False,print_out=False,one_time_additional_metadata_dict={})
                    except Exception as e:
                        self.stop_event.set()
                        self.log_print('STML Acquisition Failed: '+str(e))
                else:
                    time.sleep(1)

                self.filename_list.append(filename)

                # Check for stop criteria
                if self.stop_event.is_set():
                    print('stop event is_set')
                    break
                doZDriftCorrection = False
                doXYDriftCorrection = False
                ## Check Compensation Counters
                if self.z_drift_compensation_counter >= self.zDriftInterval.value:
                    doZDriftCorrection = True
                if self.xy_drift_compensation_counter >= self.xyDriftInterval.value:
                    doXYDriftCorrection = True

                ## XY Drift Correction
                if doXYDriftCorrection and not self._loop_test_disable_stm:
                    result = self.perform_xy_drift_correction_at_reference_position()
                    if result == 1:
                        break
                    self.xz_drift_compensation_counter = 0
                    # return to tip height for measurement and set bias
                    self.move_tip_for_measurement('experiment')
                else:
                    self.xy_drift_compensation_counter += 1

                ## Z Drift Correction
                if doZDriftCorrection and not self._loop_test_disable_stm:
                    self.perform_z_drift_stabilization_at_reference_position()
                    self.log_print('Z Drift Stabilization Complete')
                    self.move_tip_for_measurement('experiment')
                    self.z_drift_compensation_counter = 0
                else:
                    self.z_drift_compensation_counter += 1
                if not self._loop_test_disable_stm:
                    self.show_mask_selection()

                # check for pause experiment toggle
                if self.pause_event.is_set():
                    self.log_print('Experiment Paused. Waiting for Resume...')
                    self.move_tip_for_measurement('drift')
                    self.atbase.activate()
                    while self.pause_event.is_set():
                        time.sleep(1)
                    self.log_print('Continueing Experiment after drift corrections...')
                    # resume experiment
                    self.atbase.deactivate()
                    self.perform_xy_drift_correction_at_reference_position()
                    self.perform_z_drift_stabilization_at_reference_position()
                    self.move_tip_for_measurement('experiment')
                    
        except Exception as e:
            self.log_print('Experiment Script Failed ;_;')
            self.log_print('Error: '+str(e))
            # ABORT EXPERIMENT BEHAVIOUR
            self.stop_experiment('a')
            return 1
            #self.nbase.zcontroller2.#Withdraw(wait_until_finished=True)
                
        # <---------------------------------------------------------------------------------------------------------------------------------------------------------------------->
        # go to reference location and activate feedback
        self.log_print('Experiment Sequence Finished')
        self.stop_experiment('a')
        return 0
# Start LF & STML Base Class
if stml_testing_enviroment_active:
    lf = stml_testing_enviroment.lightfield()
    stmlbase = stml_testing_enviroment.STMLBase(nbase,lf,additional_metadata_dict=None)
else:
    if 'stmlbase' not in globals().keys():
        stmlbase = stml_base.STMLBase(nbase,lf,additional_metadata_dict=None)
%matplotlib widget
atbase = atomtracker_base.AtomTracker_Base(nbase)
grid_experiment = sequencer_stml_xyz_correction(nbase,lf,stmlbase,atbase,xy_correction_after_no=199,z_correction_after_no=19)

In [ ]:
# plot drift from current grid experiment
figs = plt.get_figlabels()
if 'drift' in figs:
    plt.close('drift')
fig, ax = plt.subplots(ncols=2,nrows=2,num='drift',figsize=(8,4.5))
x = [float(el[0])*1e9 for el in grid_experiment.xy_correction_array]
x_tot = np.cumsum(x)
y = [float(el[1])*1e9 for el in grid_experiment.xy_correction_array]
y_tot = np.cumsum(y)
z = [float(el[1])*1e9 for el in grid_experiment.z_correction_array]
ax[0,0].scatter(x_tot,y_tot,marker='x',alpha=.25)
[ax[0,0].text(x_tot[i],y_tot[i],i) for i in range(len(x))];
ax[0,0].set_title('XY Drift')
ax[0,0].set_xlabel('X Drift [nm]')
ax[0,0].set_ylabel('Y Drift [nm]')
ax[0,1].scatter(x,y,marker='x',alpha=.25)
[ax[0,1].text(x[i],y[i],i) for i in range(len(x))];
ax[0,1].set_title('XY Drift Correction')
ax[0,1].set_xlabel('X Drift [nm]')
ax[0,1].set_ylabel('Y Drift [nm]')
ax[1,0].plot(z)
ax[1,0].set_title('Z Drift')
ax[1,0].set_xlabel('correction number')
ax[1,0].set_ylabel('Z [nm]')
ax[1,1].plot(np.gradient(z))
ax[1,1].set_title('Z Drift Correction')
ax[1,1].set_xlabel('correction number')
fig.tight_layout(pad=1)

##### STML Experiment Sequencer: Scripted & Threaded

In [ ]:
### STML Experiment Sequencer: Scripted & Threaded ###
import ipywidgets as ipy
import time
import threading

class run_simple_experiment():

    """
    Assumes that the following modules are loaded as globas in the current namespace:
    - nbase: Nanonis Base Module
    - aquire_stml: Function to aquire a single stml spectrum
    """

    def __init__(self):
        
        # Create Widgets
        self.create_widgets()
        self.filename_list = []

    def create_widgets(self):
        self.start_button = ipy.Button(description='',icon='play',tooltip='Start Experiment',layout=ipy.Layout(width='250px',height='30px'),disabled=False)
        self.stop_button = ipy.Button(description='',icon='stop',tooltip='Stop Experiment',layout=ipy.Layout(width='250px',height='30px'),disabled=True)
        self.info_label = ipy.Label(value='Current experiment: ',layout=ipy.Layout(width='750px',height='30px'))
        self.start_button.on_click(self.start_experiment)
        self.stop_button.on_click(self.stop_experiment)
        self.widgets = ipy.HBox([self.start_button,self.stop_button,self.info_label])
        display(self.widgets)
    
    def start_experiment(self,change):
        print('Starting Experiment')
        self.start_button.disabled = True
        self.stop_button.disabled = False
        self.stop_event = threading.Event()
        self.stop_event.clear()
        self.experiment_thread = threading.Thread(target=self.experiment_thread)
        self.experiment_thread.start()

    def stop_experiment(self,change):
        print('Stopping Experiment')
        self.start_button.disabled = False
        self.stop_button.disabled = True
        self.stop_event.set()
    
    def experiment_thread(self):
        
        bias_set = None # [2.1,2.2,2.3,2.4,2.5]
        current_set = None # [100e-12,200e-12,300e-12,400e-12,500e-12]

        if bias_set is not None and current_set is not None:
            self.info_label.value = 'Only one of bias_set and current_set can be not None.'
            raise ValueError('Only one of bias_set and current_set can be not None.')
        if bias_set is None and current_set is None:
            self.info_label.value = 'One of bias_set and current_set must be not None.'
            raise ValueError('One of bias_set and current_set must be not None. You need to configure the experiment settings in this cell.')
        
        if bias_set is not None:
            bias_change = True
        else:
            bias_change = False
        
        if current_set is not None:
            current_change = True
        else:
            current_change = False

        # Get current exposure time & Calculate estimated time
        exposure_time = lf.get_exposure_time()
        if bias_set is not None:
            estimated_time = len(bias_set) * (exposure_time / 1000 + 20)/60
        elif current_set is not None:
            estimated_time = len(current_set) * (exposure_time / 1000 + 20)/60
        print('Experiment Started at: '+time.strftime("%Y:%m:%d %H:%M:%S"))
        print('ETA: ' + str(round(estimated_time,2)) + ' min')
        print('Experiment will finish at: '+time.strftime("%Y:%m:%d %H:%M:%S", time.localtime(time.time() + estimated_time*60)))

        # Loop through experimental settings
        if bias_change:
            set = bias_set
        if current_change:
            set = current_set
        for i in range(len(set)):
            if self.stop_event.is_set():
                print('self.stop_event.is_set()==True')
                self.info_label.value = 'Current experiment: Stopped'
                break
            else:
                if bias_change:
                    bias = set[i]
                    print('Setting Bias to '+str(bias)+' V')
                    self.info_label.value = 'Current experiment: Current = '+str(current)+' A and Aquiring STML Spectrum'
                    nbase.biasmodule.Set(bias)
                if current_change:
                    current = set[i]
                    print('Setting Current to '+str(current)+' A')
                    self.info_label.value = 'Current experiment: Current = '+str(current)+' A and Aquiring STML Spectrum'
                    nbase.zcontroller.SetpntSet(current)
                time.sleep(10)
                datetime_str = time.strftime("%Y%m%d_%H%M%S")
                #aquire_stml(datetime_str+'bias_dependence_pristine_'+str(bias).replace('.','_')+'V')
                filename = datetime_str+'current_dependence_pristine_'+str(round(current*1e12))+'pA'
                aquire_stml(filename)
                self.filename_list.append(filename)
                time.sleep(10)

        print('Finished')
        self.info_label.value = 'Current experiment: Finished'

experiment = run_simple_experiment()





In [ ]:
### STML Experiment Sequence: Scripted & Threaded: STML on Grid Points with XY SHIFT VECTOR CORRECTION ###
import ipywidgets as ipy
import time
import threading
import math
from skimage.registration import phase_cross_correlation
sys.path.append(r'./spmpy')
from spmpy import Spm

class sequencer_stml_xyz_correction():

    """STML Grid Experiment Sequencer with Z Drift Stabilization and (X,Y) Shift Vector Correction by Image Correlation"""

    def __init__(self,nbase,lf,stmlbase,xy_correction_after_no=10,z_correction_after_no=2,point_list=None):
        """
        Input:
        -----
        - nbase: Nanonis Base Module
        - lf: Lightfield Module
        - stmlbase: STML Base Module
        - xy_correction_after_no: Number of STML Spectra Acquisitions after which the (X,Y) Shift Vector Correction is performed
        - z_correction_after_no: Number of STML Spectra Acquisitions after which the Z Drift Correction is performed
        - point_list: List of dictionaries containing the grid points in the following format (if None, the grid points are aquired from Nanonis):
            [{'grid_point_number':0,'x':0.0,'y':0.0,'filebase':'20210901_123456_0'},...]
        """
        
        # Necessary Interaction Classes
        self.nbase = nbase
        self.lf = lf
        self.stmlbase = stmlbase
        
        # Internal Variables
        self.xy_correction_after_no = xy_correction_after_no
        self.z_correction_after_no = z_correction_after_no
        self.filename_list = []
        self.grid_params = None
        self.grid_points_params = []
        self.stm_images = []
        self.ref_location = None
        self.log_str = ''
        self.stop_event = threading.Event()
        self.stop_event.clear()

        # Create Widgets
        self.create_widgets()

        # Overwrite Grid Points if point_list is not None and has the correct format
        if point_list is not None and set(point_list[0].keys()) == {'grid_point_number','x','y','filebase'}:
            self.grid_points_params = point_list
            # Update Button Style
            self.grab_grid_points_button.style.button_color = 'lightgreen'
            self.grab_grid_points_button.description = 'Grid Points Aquired!'
            self.grab_grid_points_button.tooltip = 'Click to aquire a new grid points.'


    def create_widgets(self):
        self.grab_ref_location_button = ipy.Button(description='Acquire Reference Location',
                                            tiptool='Aquire current follow me location (x,y,V,I) to be used as ref location',
                                            layout=ipy.Layout(width='250px',height='30px'))
        self.grab_ref_location_button.on_click(self.grab_ref_location)

        self.grab_grid_points_button = ipy.Button(description='Acquire Grid Points',
                                            tiptool='Aquire (& Calculate) Grid Points Parameters from Nanonis',
                                            layout=ipy.Layout(width='250px',height='30px'))
        self.grab_grid_points_button.on_click(self.grab_grid_points)

        self.start_button = ipy.Button(description='',
                                       icon='play',
                                       tooltip='Start Experiment',
                                       layout=ipy.Layout(width='150px',height='30px'),
                                       disabled=False)
        self.start_button.on_click(self.start_experiment)

        self.stop_button = ipy.Button(description='',
                                      icon='stop',
                                      tooltip='Stop Experiment',
                                      layout=ipy.Layout(width='150px',height='30px'),
                                      disabled=True)
        self.stop_button.on_click(self.stop_experiment)
        
        self.log_textarea = ipy.Textarea(value='',layout=ipy.Layout(width='300px',height='300px'))

        self.widgets = ipy.VBox([self.grab_ref_location_button,
                                self.grab_grid_points_button,
                                ipy.HBox([self.start_button,self.stop_button]),
                                self.log_textarea])
        display(self.widgets)
    
    ### WIDGETS CALLS ###

    def grab_ref_location(self,change):
        """Reads (x,y) from follow me and I Setpoint from zcontroller, V from Bias Module and saves them as reference location"""
        x_position, y_position = nbase.followme.XYPosGet(Wait_for_newest_data=True)
        i_setpoint = nbase.zcontroller.SetpntGet()
        bias = nbase.biasmodule.Get()

        self.ref_location = {'x': x_position, 'y': y_position, 'i': i_setpoint, 'v': bias}
        print('Reference Location grabbed: ', self.ref_location)
        # Update Button Style
        self.grab_ref_location_button.style.button_color = 'lightgreen'
        self.grab_ref_location_button.description = 'Reference Location Aquired!'
        self.grab_ref_location_button.tooltip = 'Click to aquire a new reference location.'

    def grab_grid_points(self,change):
        """Reads Grid Parameters from Nanonis and calculates the grid points based on the grid parameters"""
        number_x_points, number_y_points, center_x, center_y, width, height, angle = self.nbase.pattern.GridGet()
        print('Grid Parameters aquired: ', number_x_points, number_y_points, center_x, center_y, width, height, angle)
        grid_points = self.calculate_grid_points(number_x_points, number_y_points, center_x, center_y, width, height, angle)
        filebase = time.strftime("%Y%m%d_%H%M%S")
        for i in range(len(grid_points)):
            self.grid_points_params.append({'grid_point_number':i,'x': grid_points[i][0], 'y': grid_points[i][1], 'filebase': filebase+"_"+str(i)})

        # Update Button Style
        self.grab_grid_points_button.style.button_color = 'lightgreen'
        self.grab_grid_points_button.description = 'Grid Points Aquired!'
        self.grab_grid_points_button.tooltip = 'Click to aquire a new grid points.'

    def start_experiment(self,change):
        self.log_print('Starting Experiment Sequence...')
        self.start_button.disabled = True
        self.stop_button.disabled = False
        self.experiment_thread = threading.Thread(target=self.experiment_thread)
        self.experiment_thread.start()

    def stop_experiment(self,change):
        self.log_print('Stopping Experiment Sequence...')

        # Check if Scan is running via second port
        if self.nbase.scan2.StatusGet() == 1:
            self.nbase.scan2.Action("stop")

        # Update Button Style
        self.start_button.disabled = False
        self.stop_button.disabled = True
        self.stop_event.set()

        # Print Completed File List
        print(f"{self.filename_list=}")

        # <------------------------------------------------------------------ UPDATE STOP BEHAVIOUR HERE ------------------------------------------------------------------>
        self.nbase.zcontroller2.Withdraw(wait_until_finished=True)
        # <---------------------------------------------------------------------------------------------------------------------------------------------------------------->

    ### HELPER FUNCTIONS ###
   
    def calculate_grid_points(self,number_x_points, number_y_points, center_x, center_y, width, height, angle):
        """
        Calculates the grid points based on the grid parameters.

        Input:
        -----
        - number_x_points: number of points in x direction
        - number_y_points: number of points in y direction
        - center_x: center x position of the grid in m
        - center_y: center y position of the grid in m
        - width: width of the grid in m
        - height: height of the grid in m
        - angle: angle of the grid in degrees

        Output:
        ------
        - grid_points: list of grid points (x[m],y[m],x_index,y_index)
        """

        grid_points = []
        
        # Calculate the step size for x and y directions
        step_x = width / (number_x_points)
        step_y = height / (number_y_points)
        
        # Convert the angle to radians
        angle_rad = math.radians(angle)
        
        # Calculate the starting position of the grid
        start_x = center_x - (width / 2)
        start_y = center_y - (height / 2)
        
        for i in range(number_y_points+1):
            for j in range(number_x_points+1):
                # Calculate the current position based on the grid index
                current_x = start_x + j * step_x
                current_y = start_y + i * step_y
                
                # Rotate the current position around the center point
                rotated_x = center_x + (current_x - center_x) * math.cos(angle_rad) - (current_y - center_y) * math.sin(angle_rad)
                rotated_y = center_y + (current_x - center_x) * math.sin(angle_rad) + (current_y - center_y) * math.cos(angle_rad)
                
                # Add the rotated position to the grid points
                grid_points.append((rotated_x, rotated_y,j,i))
        
        return grid_points
    
    def last_acquired_sxm(self,filepath):
        """Returns the filename of the last sxm file in the session folder."""
        files = os.listdir(filepath)
        files = [file for file in files if file.endswith('.sxm')]
        files_sorted = sorted(files, key=lambda x: os.path.getctime(os.path.join(filepath, x)))
        if len(files_sorted) > 0:
            return files_sorted[-1]
        else:
            return None

    def log_print(self,text):
        if self.log_str == '':
            self.log_str = text
        else:
            self.log_str = self.log_str + '\n' + text
        self.log_textarea.value = self.log_str

    ### Z-Drift Stabilization Function ###

    def perform_z_drift_stabilization_at_reference_position(self):
        """Go to reference location, switch on Z-Controller, set current and bias, stablize Z-Drift for 10 sec and switch off Z-Controller"""
        self.nbase.followme.XYPosSet(self.ref_location['x'],self.ref_location['y'],Wait_end_of_move=True)
        time.sleep(2)
        self.nbase.zcontroller.OnOffSet(True)
        self.nbase.zcontroller.SetpntSet(self.ref_location['i'])
        self.nbase.biasmodule.Set(self.ref_location['v'])
        time.sleep(10)
        self.nbase.zcontroller.OnOffSet(False)
    
    ### MAIN EXPERIMENT THREAD ###

    def experiment_thread(self):

        if self.ref_location is None:
            raise ValueError('Reference Location is not aquired. Please aquire the reference location first.')
        if self.grid_points_params == []:
            raise ValueError('Grid Points are not aquired. Please aquire the grid points first.')

        # Calculate the estimated experiment time
        exposure_time = self.lf.get_exposure_time()
        [x,y,w,h,angle] = self.nbase.scan.FrameGet()
        [fwd_speed,bwd_speed,fwd_line_time,bwd_line_time,const_param,speed_ratio] = self.nbase.scan.SpeedGet()
        [num_channels,channel_indexes,pixels,lines] = self.nbase.scan.BufferGet()
        time_estimate_scan = {'w':w,
                        'lines':lines,
                        'speed_ratio':speed_ratio,
                        'fwd_speed':fwd_speed,
                        'eta':w*lines*(1+speed_ratio)/fwd_speed}
        time_estimate_scan['eta'] = time_estimate_scan['w']*time_estimate_scan['lines']*(1+time_estimate_scan['speed_ratio'])/time_estimate_scan['fwd_speed']
        no_of_xy_drift_corrections = int(len(self.grid_points_params)/self.xy_correction_after_no)
        total_eta_sec = time_estimate_scan['eta']*no_of_xy_drift_corrections+len(self.grid_points_params)*exposure_time/1000
        total_eta = time.gmtime(total_eta_sec)

        self.log_print('Estimated Experiment Time: '+str(total_eta.tm_hour)+'h '+str(total_eta.tm_min)+'m '+str(total_eta.tm_sec)+'s')
        self.log_print('Start Time: '+time.strftime("%Y-%m-%d %H:%M:%S"))
        self.log_print('End Time: '+time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time() + total_eta_sec)))

        # <----------------------------------------------------------------------- UPDATE EXPERIMENT HERE ----------------------------------------------------------------------->
    
        # Perfrom intial Scan with current scan settings
        self.log_print('Intial Scan Started...')
        self.nbase.zcontroller.OnOffSet(True)
        self.nbase.scan.Action("start","down")
        self.nbase.scan.WaitEndOfScan()
        time.sleep(2)

        try:
            # Find last aquired image
            current_session_path = nbase.utilities.SessionPathGet()
            last_image_filename = self.last_acquired_sxm(current_session_path)
            if last_image_filename is None:
                self.log_print('No recent image found in the current session folder.')
                raise ValueError('No recent image found in the current session folder.')
            else:
                self.stm_images.append(last_image_filename)

            # Perform Z Drift Stabilization at Reference Position
            self.perform_z_drift_stabilization_at_reference_position()
            self.log_print('Z Drift Stabilization Complete')

            # Start Experiment Loop
            z_drift_compensation_counter = 0
            xy_drift_compensation_counter = 0

            for i in range(len(self.grid_points_params)):
                
                # Check for stop criteria
                if self.stop_event.is_set():
                    break
                
                # Go to Location and Aquire STML Spectrum
                point_x = self.grid_points_params[i]['x']
                point_y = self.grid_points_params[i]['y']
                self.nbase.followme.XYPosSet(point_x,point_y,Wait_end_of_move=True)
                time.sleep(2)
                
                # Acquire STML Spectrum
                self.log_print('Aquire STML at Grid:'+str(i+1)+'/'+str(len(self.grid_points_params)))
                filename = self.grid_points_params[i]['filebase']
                self.stmlbase.aquire_stml(filename,spe_data_return=False,print_out=False,one_time_additional_metadata_dict={})
                self.filename_list.append(filename)


                # Check for stop criteria
                if self.stop_event.is_set():
                    break

                ## Z Drift Correction
                if z_drift_compensation_counter == self.z_correction_after_no:
                    self.perform_z_drift_stabilization_at_reference_position()
                    self.log_print('Z Drift Stabilization Complete')
                    z_drift_compensation_counter = 0
                else:
                    z_drift_compensation_counter += 1

                ## XY Drift Correction
                if xy_drift_compensation_counter == self.xy_correction_after_no:
                    # Switch zcontroller only on at reference location
                    if z_drift_compensation_counter != 0:
                        self.perform_z_drift_stabilization_at_reference_position()
                    else:
                        pass

                    # Check for stop criteria
                    if self.stop_event.is_set():
                        break

                    # Perform XY Drift Correction
                    self.nbase.zcontroller.OnOffSet(True)
                    self.nbase.scan.Action("start","down")
                    self.nbase.scan.WaitEndOfScan()
                    self.nbase.zcontroller.OnOffSet(False)
                    time.sleep(2)

                    current_session_path = nbase.utilities.SessionPathGet()
                    last_image_filename = self.last_acquired_sxm(current_session_path)
                    self.stm_images.append(last_image_filename)
                    
                    # Load stm_image[-2] as image1
                    image1 = Spm(os.path.join(current_session_path,self.stm_images[-2]))
                    (chData1,chUnit) = image1.get_channel(channel='z',direction='forward',flatten=False,offset=False,zero=False);
                    height1 = image1.get_param('height')[0]*1e-9
                    width1 = image1.get_param('width')[0]*1e-9  

                    # Load stm_image[-1] as image2
                    image2 = Spm(os.path.join(current_session_path,self.stm_images[-1]))
                    (chData2,chUnit) = image2.get_channel(channel='z',direction='forward',flatten=False,offset=False,zero=False);
                    
                    self.log_print('Image Correlation: '+str(self.stm_images[-2])+' and '+str(self.stm_images[-1]))

                    # Perform Image Correlation
                    shift_measured, error, diffphase = phase_cross_correlation(chData1, chData2)
                    x_shift_nm = shift_measured[1]*(len(chData1[0])/width1)
                    y_shift_nm = shift_measured[0]*(len(chData1)/height1)

                    if abs(x_shift_nm) > 10 or abs(y_shift_nm) > 10:
                        self.log_print('XY Drift Correction: Shifted by ('+str(round(x_shift_nm,2))+' nm,'+str(round(y_shift_nm,2))+' nm)')
                        self.log_print('XY Drift Correction: Shift too large. XY Drift Compensation Failed')
                        xy_drift_compensation_counter = 0
                        z_drift_compensation_counter = 0
                        raise ValueError('XY Drift Correction: Shift too large. XY Drift Compensation Failed')

                    # Update Grid Point Parameters
                    for j in range(i,len(self.grid_points_params)):
                        self.grid_points_params[j]['x'] -= x_shift_nm
                        self.grid_points_params[j]['y'] -= y_shift_nm
                    
                    self.log_print('XY Drift Correction: Shifted by ('+str(round(x_shift_nm,2))+' nm,'+str(round(y_shift_nm,2))+' nm)')
                    xy_drift_compensation_counter = 0

                    # Go to Reference Location and Perform Z Drift Stabilization
                    self.perform_z_drift_stabilization_at_reference_position()
                    self.log_print('Z Drift Stabilization Complete')
                else:
                    xy_drift_compensation_counter += 1
                
        except Exception as e:
            self.log_print('Experiment Script Failed ;_;')
            self.log_print('Error: '+str(e))
            # ABORT EXPERIMENT BEHAVIOUR
            self.nbase.zcontroller2.Withdraw(wait_until_finished=True)
                
        # <---------------------------------------------------------------------------------------------------------------------------------------------------------------------->
                
        self.log_print('Experiment Sequence Finished')

# Start LF & STML Base Class
if stml_testing_enviroment_active:
    lf = stml_testing_enviroment.lightfield()
    stmlbase = stml_testing_enviroment.STMLBase(nbase,lf,additional_metadata_dict=None)
else:
    stmlbase = stml_base.STMLBase(nbase,lf,additional_metadata_dict=None)

experiment_stml_grid_xy_shift_correction = sequencer_stml_xyz_correction(nbase,lf,stmlbase,xy_correction_after_no=1,z_correction_after_no=1)


### Various Code

In [ ]:
### Perform Small and Large Moves ###

SMALL_MOVE_STEPS = (0, 10, 10) # (x,y,z) in steps
LARGE_MOVE_STEPS = (0, 1000, 100)

import ipywidgets as ipy
import time


def perform_move(steps):
    nbase.zcontroller.Withdraw(Wait_end_of_move=True)
    time.sleep(0.5)

    # Move in Z+ direction
    nbase.motor.StartMove('Z+', abs(steps[2]), wait_until_finished=True, group=1)

    # Move in Y direction
    if steps[1] > 0:
        nbase.motor.StartMove('Y+', abs(steps[1]), wait_until_finished=True, group=1)
    elif steps[1] < 0:
        nbase.motor.StartMove('Y-', abs(steps[1]), wait_until_finished=True, group=1)
    time.sleep(0.5)

    # Move in X direction
    if steps[0] > 0:
        nbase.motor.StartMove('X+', abs(steps[0]), wait_until_finished=True, group=1)
    elif steps[0] < 0:
        nbase.motor.StartMove('X-', abs(steps[0]), wait_until_finished=True, group=1)
    time.sleep(0.5)

    nbase.autoapproach.Approach(Wait_end_of_move=True)

def on_large_move_button_click(b):
    perform_move(LARGE_MOVE_STEPS)

def on_small_move_button_click(b):
    perform_move(SMALL_MOVE_STEPS)

button_small_move_description = f'Perform Small Move ({SMALL_MOVE_STEPS[0]}x, {SMALL_MOVE_STEPS[1]}y, {SMALL_MOVE_STEPS[2]}z)'
button_small_move = ipy.Button(description=button_small_move_description, layout=ipy.Layout(width='300px', height='30px'),icon='arrows-alt')
button_small_move.on_click(on_small_move_button_click)

button_large_move_description = f'Perform Large Move ({LARGE_MOVE_STEPS[0]}x, {LARGE_MOVE_STEPS[1]}y, {LARGE_MOVE_STEPS[2]}z)'
button_large_move = ipy.Button(description=button_large_move_description, layout=ipy.Layout(width='300px', height='30px'),icon='arrows-alt')
button_large_move.on_click(on_large_move_button_click)

display(ipy.VBox([button_small_move, button_large_move]))

In [ ]:
### Flip EOS Mirror ###

def flip_EOS_mirror(nbase,flip_up):
    """Flips the EOS mirror."""
    # Check if Mirror is up or down
    TTLVals = nbase.diglines.TTLValGet(port=2)
    EOS_mirror_bool = TTLVals[1]
    # EOS_mirror_bool == 0 == 'UP', EOS_mirror_bool == 1 == 'DOWN'
    print(f'Current EOS Mirror State: {"UP" if EOS_mirror_bool == 0 else "DOWN"}')

    # Flip the mirror if needed

    if flip_up and EOS_mirror_bool == 0:
        print('Flipping EOS: already Up')
    elif not flip_up and EOS_mirror_bool == 1:
        print('Flipping EOS: already Down')
    else:
        port = 2
        digital_lines = 1
        pulse_width = 0.15
        pulse_pause = 1
        number_of_pulses = 1
        wait_until_finished = True
        nbase.diglines.Pulse(port=port,digital_lines=digital_lines,pulse_width=pulse_width,pulse_pause=pulse_pause,number_of_pulses=number_of_pulses,wait_until_finished=wait_until_finished)
        print('Flipping EOS Mirror')

    EOS_mirror_bool_str = 'UP' if EOS_mirror_bool == 0 else 'DOWN'
    return EOS_mirror_bool_str

EOS_mirror_bool_str = flip_EOS_mirror(nbase,flip_up=False)
print(EOS_mirror_bool_str)

In [ ]:
### Get Power Measurement with Ophir Power Meter ###

import win32com.client
import time
import numpy as np
import subprocess

def does_process_exist(process_name):
    """Checks if a process with the name process_name is running."""
    progs = str(subprocess.check_output('tasklist'))
    if process_name in progs:
        return True
    else:
        return False

def get_power_measurement():
    # Check if Starlab Software is open:
    process_name = "StarLab 3.93.exe"
    process_bool = does_process_exist(process_name)
    if process_bool:
        print(f"{process_name} is running. If powermeter is already connected , this script will not work.")
        return None
    

    # Connect to the power meter
    OphirCOM = win32com.client.Dispatch("OphirLMMeasurement.CoLMMeasurement")
    powermeter = OphirCOM.OpenUSBDevice(OphirCOM.ScanUSB()[0])  # Open the first found device

    # Recieve the data
    OphirCOM.StartStream(powermeter, 0)
    time.sleep(0.5)  # Allow some time for the measurement to complete
    data = OphirCOM.GetData(powermeter, 0)
    power = np.mean(data[0])

    # Close the connection
    OphirCOM.StopAllStreams()
    OphirCOM.CloseAll()

    return power

power = get_power_measurement()
print("Laser Power in W:", power)
print("Laser Power in mW:", round(power * 1e3, 2))

In [ ]:
### Control VIS HWP angle ###

# region: get and set VIS HWP angle functions

import time
import numpy as np
import libximc.highlevel as ximc

def get_current_position_in_degrees():
    """
    Get the current position of the motor in degrees.

    Returns
    -------
    tuple: (position in degrees, (p, up))
    """

    vis_HWP_uri = 'xi-net://192.168.0.213/0008C51'
    device_uri = vis_HWP_uri
    axis = ximc.Axis(device_uri)
    axis.open_device()

    position = axis.get_position() # This is relative to the HOME position

    # Convert position to degrees
    gear_ratio = 80
    p = position.Position
    up = position.uPosition
    partial_p = up / 256
    current_zero_offset = 1.5  # degrees offset to set zero position
    degrees = (p + partial_p) / gear_ratio - current_zero_offset

    print(f"Position in degrees: {degrees}")

    axis.close_device()

    return degrees, (p, up)  # return degrees and raw position

def set_position_in_degrees(degrees):
    """
    Set the position of the motor in degrees.

    Input
    -----
    degrees: position in degrees
    """

    vis_HWP_uri = 'xi-net://192.168.0.213/0008C51'
    device_uri = vis_HWP_uri
    axis = ximc.Axis(device_uri)
    axis.open_device()

    degrees_target = degrees

    current_zero_offset = 1.5  # degrees offset to set zero position
    degrees_target = degrees_target + current_zero_offset  # add offset to set zero position

    gear_ratio = 80
    degrees = degrees_target * gear_ratio
    d = int(degrees // 1)
    ud = degrees % 1
    up = int(ud*256)
    p = d

    axis.command_move(p, up)

    axis.command_wait_for_stop(30)  # wait for the stage to stop

    axis.close_device()

# endregion

In [ ]:
get_current_position_in_degrees()

In [ ]:
target_angle = 19.75

print('initial:',get_current_position_in_degrees())
print('setting new position:',target_angle)
set_position_in_degrees(target_angle)
print('new position:',get_current_position_in_degrees())